In [ ]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime,date
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
plt.style.use('seaborn-darkgrid')
from math import fsum
from varname import nameof
from scipy import stats
from scipy.stats import pearsonr,spearmanr
import os
import myfunctions as mf
from importlib import reload
from pandas.api.types import is_numeric_dtype
import warnings
warnings.filterwarnings('ignore')
import random

#Define some lists/arrays

#https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/tech_note/Ecosystem/CLM50_Tech_Note_Ecosystem.html#id17
depth  = np.array([0.02, 0.04, 0.06, 0.08,0.12,0.16,0.20,0.24,0.28,0.32,0.36,0.40,0.44,0.54,0.64,0.74,0.84,0.94,1.04,1.14,2.39,4.676,7.635,11.140,15.115]) #layer thicknesses
node_z = [0.01,0.04,0.09,0.16,0.26,0.40,0.587,0.80,1.06,1.36,1.70,2.08,2.50,2.99,3.58,4.27,5.06,5.95,6.94,8.03,9.795,13.328,19.483,28.871,41.998] #depth of midpoint of layers
depth_30 = np.array([0.02,0.04,0.06,0.08,0.10]) #layer depths to multiply with concentrations
depth_50 = np.array([0.02,0.04,0.06,0.08,0.12,0.16,0.02]) #[meters]

sec_pr_day = 60*60*24

C_pool_names = ['LITm', 'LITs', 'SAPb', 'SAPf', 'EcM','AM', 'SOMp', 'SOMa', 'SOMc'] 
N_pool_names = ['N_LITm', 'N_LITs', 'N_SAPb', 'N_SAPf', 'N_EcM', 'N_AM','N_SOMp', 'N_SOMa', 'N_SOMc', 'N_NH4_sol','N_NH4_sorp','N_NO3']

PFT_names = ["not_vegetated                           ",
    "needleleaf_evergreen_temperate_tree     ",
    "needleleaf_evergreen_boreal_tree        ",
    "needleleaf_deciduous_boreal_tree        ",
    "broadleaf_evergreen_tropical_tree       ",
    "broadleaf_evergreen_temperate_tree      ",
    "broadleaf_deciduous_tropical_tree       ",
    "broadleaf_deciduous_temperate_tree      ",
    "broadleaf_deciduous_boreal_tree         ",
    "broadleaf_evergreen_shrub               ",
    "broadleaf_deciduous_temperate_shrub     ",
    "broadleaf_deciduous_boreal_shrub        ",
    "c3_arctic_grass                         ",
    "c3_non-arctic_grass                     ",
    "c4_grass                                ",]

PFT_names=[i.strip() for i in PFT_names]

temp_colors = ["blue","red"]
prec_colors = ["goldenrod","lightseagreen"]

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

obs_cols = ['CLFH', 'C30', 'cstock', 'c100min','LFHDYP', 'PROFDYPC', 'NLFH', 'nstock',  'n100min','ALTITUDE','PBONITET', 'VEKSTSES', 'DRAINAGE',"MAP","MAT"]
pod_cols = ['CLFH', 'C30', 'cstock', 'c100min','LFHDYP', 'PROFDYPC', 'NLFH', 'nstock',  'n100min','ALTITUDE','PBONITET', 'VEKSTSES', 'DRAINAGE',"MAP","MAT"]
dec_cols = ['TOTC', 'TOTC_input', 'HR_flux', 'c30_SOM_tot', 'c30_tot','depth','SAP', 'MYC', 'FBratio','pct_microbes', 'TOTLITC', 'TOTSOMC', 'TOTN', 'TOTLITN', 'TOTSOMN','NH4_sol', 'NH4_sorp', 'NO3']
CLM_cols = ['TOTSOMC', 'TOTSOMN', 'TOTLITC','TOTLITN', 'HR', 'NPP', 'LITFALL', 'n_bedrock', 'c30_tot', 'TOTN', 'TOTC',"MAT","MAP"]

pd.set_option('display.max_columns', 500)

### Import data

In [ ]:
def get_observations(list_of_site_numbers):

    df = pd.read_excel("/home/ecaas/Desktop/jordprofiler/CNstocks2016_HAdW_updated_LTS9august21.xlsx", sheet_name="Sheet1") #Get data
    komm = pd.read_excel("/home/ecaas/Desktop/jordprofiler/Kommuneklassifisering_1994.xlsx", sheet_name="Sheet1") #Get Kommunenavn
    all_sites = df[1:]  # All lines containing values (all but first line)
    info = df.iloc[0]  # First line contains more info

    # Change dtype to approproate types
    for c in all_sites.columns:
        if all_sites.loc[:, c].dtype == object:
            if type(all_sites.loc[:, c].iloc[0]) != str:
                all_sites.loc[:, c] = pd.to_numeric(all_sites.loc[:, c])
            else:
                all_sites.loc[:, c] = all_sites.loc[:, c].astype('category')

    # Add kommune (county) as a column to the data frame "all_sites"
    kommune = []
    for knr in all_sites.KOMM:
        if knr not in komm.loc[:, 'Kommune-nr.'].values:
            kommune.append('NO MATCH')
        else:
            for snr in komm.loc[:, 'Kommune-nr.']:
                if knr == snr:
                    kommune.append(
                        komm[komm['Kommune-nr.'] == snr]['Kommunenavn'].values[0])
    all_sites.loc[:, 'Kommunenavn'] = kommune

    # Chosen locations for decomp development 
    sites = list_of_site_numbers
    sites_df = all_sites.loc[all_sites['PROFNR'].isin(sites)].reset_index()

    for i in range(len(sites_df)):
        site = ('NR'+str(sites_df.loc[i, 'PROFNR']) + '_'+sites_df.loc[i, 'Kommunenavn']).strip()
        sites_df.loc[i, 'SITE'] = site
        obs = sites_df.loc[:, ['SITE','PROFNR', 'CLFH', 'C30', 'C50', 'cs100', 'cstock', 'c100min', 'COMratio', 'LFHDYP', 'PROFDYPC', 'NLFH', 'nstock', 'ns100', 'n100min', 'NOMratio', 'CNFOREST', 'KOORDINA','MAP', 'MAT', 'TREG', 'VEGETASJ', 'ALTITUDE', 'PBONITET', 'VEKSTSES', 'DRAINAGE', 'FAOTex', 'CanTex', "CANADIAN", "VEGTYP", "FID", "BEST_TRE"]].sort_values(by='SITE')

    obs.loc[:, "origin"] = "obs"

    for col in obs[['LFHDYP', 'PROFDYPC']].columns:  # convert observed depths to meters
        obs.loc[:, col] = obs.loc[:, col]/100.0

    for col in obs[['cstock', 'nstock', 'C50', 'C30', 'MAP', 'MAT', "cs100", "ns100", "c100min", "n100min", "CLFH", "NLFH"]].columns:  # Round to two decimals
        obs.loc[:, col] = round(obs.loc[:, col], 2)

    english_var_names={"VEKSTSES":"growing_season","PROFDYPC":"tot_depth","LFHDYP":"depth_LFH_layer","PBONITET":"quality_class"}
    obs.rename(columns = english_var_names,inplace=True)
    return obs, info

In [ ]:
#Create dataframe containing OBS data
chosen_sites=['32258','32246' ,'31984','31976','32379','32441','32032','32124','32288','31463','31464','32087','31652','31881','32182','32249','32485','31682','31578','32361',\
        '31585','32085','31714','31650','31519','32438','32409','32404','32374','32139','32103','32088','31997',\
        '31513','31539','31767','31780','31941','31906','31927','31461','32174','32173','32170','31883','31801','31799','31579','31577','31677']
obs_subset, info = get_observations(chosen_sites)

#Replace Norwegian letters
obs_subset.replace(to_replace="NR32258_Målselv",value="NR32258_Maaselv",inplace =True)
obs_subset.replace(to_replace="NR32085_Skjåk",value="NR32085_Skjaak",inplace =True)
obs_subset.replace(to_replace="NR31780_Våler",value="NR31780_Vaaler",inplace =True)
obs_subset.replace(to_replace="NR31714_Flå",value="NR31714_Flaa",inplace =True)
obs_subset.replace(to_replace="NR31941_Røyrvik",value="NR31941_Roeyrvik",inplace =True)
obs_subset.replace(to_replace="NR31682_Tysvær",value="NR31682_Tysvar",inplace =True)
obs_subset.replace(to_replace="NR32032_Vestre  Toten",value="NR32032_VestreToten",inplace =True)
obs_subset.replace(to_replace="NR32246_Sør-Varanger",value="NR32246_SoerVaranger",inplace =True)

obs_subset.head()

In [ ]:
#create a dataframe containing all podzolic sites in the database:
df = pd.read_excel("/home/ecaas/Desktop/jordprofiler/CNstocks2016_HAdW_updated_LTS9august21.xlsx", sheet_name="Sheet1")
alle = df[1:]
all_sites = alle.PROFNR.values

all_obs,info_all = mf.get_observations(list(all_sites))
all_obs = all_obs.sort_values(by="SITE")

pod1= all_obs[all_obs.CANADIAN == "Podzol"]
pod2 = all_obs[all_obs.CANADIAN == "Podzoli"]

podzols = pod1.append(pod2)

del pod1,pod2,alle,df,info_all 


#### Load CLM data

In [ ]:
#Import CLM results into xarrays
path='/home/ecaas/Documents/all_1988-1992/'
directory = os.fsencode(path)
CLM_xarray=[]
for file in os.listdir(directory):
    filename = os.fsdecode(file) 
    name = filename[:-22]
    if name[0:2] != 'NR':
        name='NR'+name+'_CLM'
    else:
        name=name+'_CLM' 
    globals()[name] = xr.load_dataset(path+filename)
    globals()[name].attrs['site_name'] = name[:-4]
    CLM_xarray.append(globals()[name])

CLM_xarray =  sorted(CLM_xarray,key=lambda x: x.site_name)


In [ ]:
def make_CLM_df(xarray_list):

    column_names = ['SITE','RAIN', 'SNOW', 'TSA', 'TOTSOMC','TOTSOMN', 'TOTLITC', 'TOTLITN']
    mean_df = pd.DataFrame(columns=column_names)
    mean_df['MAP'] = np.nan
    hr = []
    mean_df["SITE"] = (mean_df.SITE).astype("category")

    # layer depths to multiply with concentrations
    depth_30 = np.array([0.02, 0.04, 0.06, 0.08, 0.10])
    depth_50 = np.array([0.02, 0.04, 0.06, 0.08, 0.12, 0.16, 0.02])  # [meters]
    sec_pr_day = 60*60*24

    for i in range(len(xarray_list)):  # loop through all xarrays (one for each site)
        val = xarray_list[i][['RAIN', 'SNOW', 'TSA', 'TOTSOMC','TOTSOMN', 'TOTLITC', 'TOTLITN', "HR", "NPP", "LITFALL"]]
        # mean over total time period for the vars in the line above
        val = val.mean(dim="time")
        # convert from xarray to pd dataframe (row)
        val = val.to_dataframe().reset_index(drop=True)
        # add site name to val (which is a dataframe row)
        val['SITE'] = [xarray_list[i].site_name]
        # add row to main dataframe
        mean_df = mean_df.append(val, ignore_index=True)

        col = xarray_list[i][['SOIL1C_vr', 'SOIL2C_vr','SOIL3C_vr', 'LITR1C_vr', 'LITR2C_vr', 'LITR3C_vr']]
        col = col.mean(dim="time")  # Mean concentration for each layer
        col = col.to_dataframe().reset_index(drop=True)

        # get correct row in dataframe mean_df
        t = mean_df.loc[mean_df['SITE'] == xarray_list[i].site_name]

        for var in col.columns:  # loop variables
            # multiply with relevant depths to get mass (gC/m2)
            col30 = col[var][0:len(depth_30)]*depth_30
            col50 = col[var][0:len(depth_50)]*depth_50
            sum30 = col30.sum()  # Sum of mass in the relevant layers
            sum50 = col50.sum()
            mean_df.loc[t.index, 'c50_%s' % var[0:5]] = round(
                sum50, 2)  # add to the row where site name matches
            mean_df.loc[t.index, 'c30_%s' % var[0:5]] = round(sum30, 2)

        ##########Nitrogen#######################
        colN = xarray_list[i][['SOIL1N_vr', 'SOIL2N_vr','SOIL3N_vr', 'LITR1N_vr', 'LITR2N_vr', 'LITR3N_vr']]
        colN = colN.mean(dim="time")  # Mean concentration for each layer
        colN = colN.to_dataframe().reset_index(drop=True)

        # get correct row in dataframe mean_df
        t = mean_df.loc[mean_df['SITE'] == xarray_list[i].site_name]

        for var in colN.columns:  # loop variables
            # multiply with relevant depths to get mass (gC/m2)
            colN30 = colN[var][0:len(depth_30)]*depth_30
            colN50 = colN[var][0:len(depth_50)]*depth_50
            sum30 = colN30.sum()  # Sum of mass in the relevant layers
            sum50 = colN50.sum()
            mean_df.loc[t.index, 'n50_%s' % var[0:5]] = round(
                sum50, 4)  # add to the row where site name matches
            mean_df.loc[t.index, 'n30_%s' % var[0:5]] = round(sum30, 4)
            ################################


        # Put shallowest bedrock layer (nbedrock) in dataframe mean_df
        nbedrock = xarray_list[i].nbedrock.values
        mean_df.loc[t.index, 'n_bedrock'] = nbedrock

        # Put mean annual precipitation from simulations in mean_df
        # Add RAIN and SNOW to get total precip
        daily_precip = (xarray_list[i].SNOW + xarray_list[i].RAIN)*sec_pr_day
        # Find annual precip for each year
        yearly_precip = daily_precip.groupby('time.year').sum()
        # Find mean annual precip for the years 1988-1992 (drop 1993)
        precip = yearly_precip[:-1].mean().values
        mean_df.loc[t.index, 'MAP'] = round(float(precip), 1)

        ###########Carbon############################
        sum_soil50 = mean_df.loc[t.index, 'c50_SOIL1'] + mean_df.loc[t.index, 'c50_SOIL2'] + mean_df.loc[t.index, 'c50_SOIL3']
        mean_df.loc[t.index, 'c50_SOIL_tot'] = sum_soil50

        sum_soil30 = mean_df.loc[t.index, 'c30_SOIL1'] + mean_df.loc[t.index, 'c30_SOIL2']+mean_df.loc[t.index, 'c30_SOIL3']
        mean_df.loc[t.index, 'c30_SOIL_tot'] = sum_soil30

        sum_tot50 = mean_df.loc[t.index, 'c50_SOIL1']+mean_df.loc[t.index, 'c50_SOIL2']+mean_df.loc[t.index, 'c50_SOIL3'] + \
                    mean_df.loc[t.index, 'c50_LITR1']+mean_df.loc[t.index,'c50_LITR2']+mean_df.loc[t.index, 'c50_LITR3']
        mean_df.loc[t.index, 'c50_tot'] = sum_tot50

        sum_tot30 = mean_df.loc[t.index, 'c30_SOIL1']+mean_df.loc[t.index, 'c30_SOIL2']+mean_df.loc[t.index, 'c30_SOIL3'] + \
                    mean_df.loc[t.index, 'c30_LITR1']+mean_df.loc[t.index,'c30_LITR2']+mean_df.loc[t.index, 'c30_LITR3']
        mean_df.loc[t.index, 'c30_tot'] = sum_tot30

        ###########Nitrogen############################
        sumN_soil50 = mean_df.loc[t.index, 'n50_SOIL1'] + \
            mean_df.loc[t.index, 'n50_SOIL2']+mean_df.loc[t.index, 'n50_SOIL3']
        mean_df.loc[t.index, 'n50_SOIL_tot'] = sumN_soil50

        sumN_soil30 = mean_df.loc[t.index, 'n30_SOIL1'] + \
            mean_df.loc[t.index, 'n30_SOIL2']+mean_df.loc[t.index, 'n30_SOIL3']
        mean_df.loc[t.index, 'n30_SOIL_tot'] = sumN_soil30

        sumN_tot50 = mean_df.loc[t.index, 'n50_SOIL1']+mean_df.loc[t.index, 'n50_SOIL2']+mean_df.loc[t.index, 'n50_SOIL3'] + \
            mean_df.loc[t.index, 'n50_LITR1']+mean_df.loc[t.index,
                                                          'n50_LITR2']+mean_df.loc[t.index, 'n50_LITR3']
        mean_df.loc[t.index, 'n50_tot'] = sumN_tot50

        sumN_tot30 = mean_df.loc[t.index, 'n30_SOIL1']+mean_df.loc[t.index, 'n30_SOIL2']+mean_df.loc[t.index, 'n30_SOIL3'] + \
            mean_df.loc[t.index, 'n30_LITR1']+mean_df.loc[t.index,
                                                          'n30_LITR2']+mean_df.loc[t.index, 'n30_LITR3']
        mean_df.loc[t.index, 'n30_tot'] = sumN_tot30
        #############################


    mean_df.loc[:, "origin"] = "CLM"

    # convert mean_df temperature to celsius
    mean_df.loc[:, "TSA_degC"] = round(mean_df.TSA-273.15, 1)

    mean_df["TOTN"] = mean_df.TOTSOMN+mean_df.TOTLITN
    mean_df["TOTC"] = mean_df.TOTSOMC+mean_df.TOTLITC
    mean_df["HR"]  = mean_df.HR*60*60 

    for col in mean_df[['TOTSOMC', 'TOTSOMN', 'TOTLITC', 'c50_tot', 'c30_tot', 'TOTC', 'TOTN',
                        'c50_SOIL1', 'c30_SOIL1', 'c50_SOIL2', 'c30_SOIL2', 'c50_SOIL3', 'c30_SOIL3',
                        'c50_LITR1', 'c30_LITR1', 'c50_LITR2', 'c30_LITR2', 'c50_LITR3', 'c30_LITR3', 
                        'c50_SOIL_tot', 'c30_SOIL_tot',]].columns:  # convert mean_df content to kg/m2
        mean_df.loc[:, col] = round(mean_df.loc[:, col]*1e-3, 2)

    for col in mean_df[['n50_SOIL_tot','n30_SOIL_tot','n50_tot','n30_tot']].columns:
        mean_df.loc[:,col]=round(mean_df.loc[:, col]*1e-3, 4)

    mean_df.loc[:, "TOTLITN"] = round(mean_df.loc[:, "TOTLITN"]*1e-3, 4)

    mean_df = mean_df.sort_values(by="SITE")
    mean_df.reset_index(inplace=True)
    mean_tmp = mean_df[['c50_SOIL1', 'c30_SOIL1', 'c50_SOIL2', 'c30_SOIL2', 'c50_SOIL3', 'c30_SOIL3', 'c50_LITR1',
                        'c30_LITR1', 'c50_LITR2', 'c30_LITR2', 'c50_LITR3', 'c30_LITR3', 'c50_SOIL_tot', 'c30_SOIL_tot', 'SITE',
                        'n50_SOIL1', 'n30_SOIL1','n50_SOIL2', 'n30_SOIL2', 'n50_SOIL3', 'n30_SOIL3', 'n50_LITR1',
                        'n30_LITR1', 'n50_LITR2', 'n30_LITR2', 'n50_LITR3', 'n30_LITR3']]
    mean_df.drop(['c50_SOIL1', 'c30_SOIL1', 'c50_SOIL2', 'c30_SOIL2', 'c50_SOIL3', 'c30_SOIL3', 'c50_LITR1', 'c30_LITR1',
                 'c50_LITR2', 'c30_LITR2', 'c50_LITR3', 'c30_LITR3', 'c50_SOIL_tot', 'c30_SOIL_tot','n50_SOIL1', 'n30_SOIL1',
       'n50_SOIL2', 'n30_SOIL2', 'n50_SOIL3', 'n30_SOIL3', 'n50_LITR1',
       'n30_LITR1', 'n50_LITR2', 'n30_LITR2', 'n50_LITR3', 'n30_LITR3'], axis=1, inplace=True)

    return mean_df, mean_tmp

In [ ]:
#dataframes with info (and extra info):
CLM_df, CLM_df_tmp = make_CLM_df(CLM_xarray)

CLM_df.loc[:,"MAT"]=CLM_df["TSA_degC"]
CLM_df.drop(columns="index", axis = 1, inplace=True)
CLM_df.head()


#### Load data for time period

In [ ]:
#Function for importing MIMICS+ results and store them as xarrays
def import_decomp(path_of_directory,dir_list,file_ext,date_bool):
    site_data=[]
    decomp_names=[]
    dirs = dir_list
 

    for folder in dirs:
        ext = (folder+file_ext)
        for files in os.listdir(path_of_directory+folder):
            if files.endswith(ext):
                filename = os.fsdecode(files)  
                name = filename[:-len(ext)-1]
                if name[0:2] != 'NR':
                    name='NR'+name#+folder
                else:
                    name = name #+folder
                decomp_names.append(name)
                globals()[name+"_"+folder+file_ext[0:-3]] = xr.load_dataset(path_of_directory+folder+"/"+filename)

                globals()[name+"_"+folder+file_ext[0:-3]].attrs['site_name'] = name
                globals()[name+"_"+folder+file_ext[0:-3]].attrs['run_name'] = folder
                if date_bool:
                    mf.add_date_dim(globals()[name+"_"+folder+file_ext[0:-3]])

                site_data.append(globals()[name+"_"+folder+file_ext[0:-3]])
            else:
                continue
        site_data =  sorted(site_data,key=lambda x: x.site_name)
        decomp_names = sorted(decomp_names)
        #decomp_names: list of name strings
        #site_data: list of xarrays
    return decomp_names,site_data


In [ ]:
#List of strings, list of xarrays:
decomp_names,decomp_data = import_decomp("/home/ecaas/paper_version/soil_decomp/results/",["default_run"],"_to1992.nc",True)
decomp_df,decomp_tmp_df = mf.make_decomp_df(decomp_data)

#### Describe the different datasets:

In [ ]:
podzols[pod_cols].describe()


In [ ]:
obs_subset.describe()

In [ ]:
decomp_df.describe()

In [ ]:
CLM_df.describe()

### categorize datasets 
Divide in 2 equally sized bins for each subset

Add a column containing mean soil water down to 10 cm from CLM simulations. Used as an alternative way of dividing dataset after moisture/precipitation

In [ ]:
sw=[]
name=[]
for i in range(len(CLM_xarray)):
    name.append(CLM_xarray[i].site_name)
    sw.append(float(CLM_xarray[i].SOILWATER_10CM.mean()))
    #sw.append(float(CLM_xarray[i].H2OSOI.sel(levsoi=slice(0,node_z[int(CLM_xarray[i].nbedrock)])).mean())*100)
data = pd.DataFrame({"SITE":name,"SOILWATER":sw})
#Sort by site:
data = data.sort_values("SITE").reset_index(drop=True)

#Add column to dataframes (that are already sorted by SITE)
CLM_df["SOILWATER"] = data["SOILWATER"]
decomp_df["SOILWATER"] = data["SOILWATER"]
obs_subset["SOILWATER"] = data["SOILWATER"]

del data,sw,name

In [ ]:
SOILWATER_cat = ["drier_soil" for i in range(25)] + ["wetter_soil" for i in range(25)]

def add_soilwater_cat(df):
    ##Ny df sorted by SOILWATER
    sorted_SOILWATER=df.sort_values(by="SOILWATER").reset_index()

    ##Add category column SOILWATER_cat to the new df
    sorted_SOILWATER["SW_cat"] = SOILWATER_cat
    df["cat_SW"] = sorted_SOILWATER.reindex(sorted_SOILWATER["index"]).sort_values("SITE").SW_cat.values
    del sorted_SOILWATER
add_soilwater_cat(decomp_df)
add_soilwater_cat(obs_subset)
add_soilwater_cat(CLM_df)


In [ ]:
#Add MAT and MAP from CLM dataframe to MIM dataframe bc. they use the same forcing data. 
decomp_df["MAP"]=CLM_df.MAP
decomp_df["MAT"]=CLM_df.MAT

MAT_cat = ["cooler" for i in range(25)] + ["warmer" for i in range(25)]
MAP_cat = ["drier" for i in range(25)] + ["wetter" for i in range(25)] 

def add_cat(df,variable,cat_list,cat_name):
    ##Ny df sorted by MAT
    sorted_df=df.sort_values(by=variable).reset_index()

    ##Add category cat_MAT to the new df
    sorted_df[cat_name] = cat_list
    df[cat_name] = sorted_df.sort_values("SITE")[[cat_name]].values
    df[cat_name] = sorted_df.reindex(sorted_df["index"]).sort_values("SITE")[[cat_name]].values
    del sorted_df

#add MAP category
add_cat(decomp_df,"MAP",MAP_cat,"MAP_cat")
add_cat(CLM_df,"MAP",MAP_cat,"MAP_cat")
add_cat(obs_subset,"MAP",MAP_cat,"MAP_cat")

#add MAT category
add_cat(decomp_df,"MAT",MAT_cat,"MAT_cat")
add_cat(CLM_df,"MAT",MAT_cat,"MAT_cat")
add_cat(obs_subset,"MAT",MAT_cat,"MAT_cat")

In [ ]:
#All podzols
clean_pod = podzols[podzols['MAP'].notna()]
clean_pod = clean_pod[clean_pod['MAT'].notna()].reset_index(drop=True)
podz_wo_subset = clean_pod[~clean_pod['PROFNR'].isin(obs_subset['PROFNR'])] #All podzols except those used for model simulations

MAT_median = str(podz_wo_subset.MAT.median())
MAP_median = str(podz_wo_subset.MAP.median())

cold = podz_wo_subset.query("MAT<="+MAT_median)
warm = podz_wo_subset.query("MAT>"+MAT_median)
cold.loc[:,"MAT_cat"] = ["cooler" for i in range(len(cold))]
warm.loc[:,"MAT_cat"] = ["warmer" for i in range(len(warm))]
cold=cold.reset_index(drop=True)
warm=warm.reset_index(drop=True)

podz_wo_subset = cold.merge(warm,how="outer").sort_values("SITE").reset_index(drop=True)

t=[]
for val in podz_wo_subset.MAP:
    if val > float(MAP_median):
        t.append("wetter")
    else:
        t.append("drier")
podz_wo_subset.loc[:,"MAP_cat"] = t
podz_wo_subset.head()


##### Add profile stocks as columns

In [ ]:
#Total (including LFH layer)
podz_wo_subset.loc[:,"stock30"]      = podz_wo_subset.C30
podz_wo_subset.loc[:,"stock30_50"]   = (podz_wo_subset.C50-podz_wo_subset.C30)
podz_wo_subset.loc[:,"stock50_100"]   = (podz_wo_subset.cs100-podz_wo_subset.C50)

#Mineral soil only (excluding  LFH layer)
podz_wo_subset.loc[:,"stock_min30"] = (podz_wo_subset.C30-podz_wo_subset.CLFH)
podz_wo_subset.loc[:,"stock_min30_50"]    = (podz_wo_subset.C50-podz_wo_subset.C30)
podz_wo_subset.loc[:,"stock_min50_100"]    = podz_wo_subset.stock50_100

#Total (including LFH layer)
podzols.loc[:,"stock30"]      = podzols.C30
podzols.loc[:,"stock30_50"]   = (podzols.C50-podzols.C30)
podzols.loc[:,"stock50_100"]   = (podzols.cs100-podzols.C50)

#Mineral soil only (excluding  LFH layer)
podzols.loc[:,"stock_min30"] = (podzols.C30-podzols.CLFH)
podzols.loc[:,"stock_min30_50"]    = (podzols.C50-podzols.C30)
podzols.loc[:,"stock_min50_100"]    = podzols.stock50_100

#Total (including LFH layer)
obs_subset.loc[:,"stock30"]      = obs_subset.C30
obs_subset.loc[:,"stock30_50"]   = (obs_subset.C50-obs_subset.C30)
obs_subset.loc[:,"stock50_100"]   = (obs_subset.cs100-obs_subset.C50)

#Mineral soil only (excluding  LFH layer)
obs_subset.loc[:,"stock_min30"] = (obs_subset.C30-obs_subset.CLFH)
obs_subset.loc[:,"stock_min30_50"]    = (obs_subset.C50-obs_subset.C30)
obs_subset.loc[:,"stock_min50_100"]    = obs_subset.stock50_100

#Total (including litter pools)
CLM_df.loc[:,"stock30"]      = round(CLM_df.c30_tot,2)
CLM_df.loc[:,"stock30_50"]   = round((CLM_df.c50_tot-CLM_df.c30_tot),2)
CLM_df.loc[:,"stock50_100"]   = round((CLM_df.TOTC-CLM_df.c50_tot),2)

#Mineral soil only (excluding litter pools)
CLM_df.loc[:,"stock_min30"] = round((CLM_df_tmp.c30_SOIL1+CLM_df_tmp.c30_SOIL2+CLM_df_tmp.c30_SOIL3),2)
CLM_df.loc[:,"stock_min30_50"]    = round(((CLM_df_tmp.c50_SOIL1+CLM_df_tmp.c50_SOIL2+CLM_df_tmp.c50_SOIL3)-(CLM_df_tmp.c30_SOIL1+CLM_df_tmp.c30_SOIL2+CLM_df_tmp.c30_SOIL3)),2)
CLM_df.loc[:,"stock_min50_100"]    = round((CLM_df.TOTSOMC-(CLM_df_tmp.c50_SOIL1+CLM_df_tmp.c50_SOIL2+CLM_df_tmp.c50_SOIL3)),2)

#Total (including litter pools)
CLM_df.loc[:,"Nstock30"]      = round(CLM_df.n30_tot,2)
CLM_df.loc[:,"Nstock30_50"]   = round((CLM_df.n50_tot-CLM_df.n30_tot),4)
CLM_df.loc[:,"Nstock50_100"]   = round((CLM_df.TOTN-CLM_df.n50_tot),2)

#Mineral soil only (excluding litter pools)
CLM_df.loc[:,"Nstock_min30"] = round((CLM_df_tmp.n30_SOIL1+CLM_df_tmp.n30_SOIL2+CLM_df_tmp.n30_SOIL3),2)
CLM_df.loc[:,"Nstock_min30_50"]    = round(((CLM_df_tmp.n50_SOIL1+CLM_df_tmp.n50_SOIL2+CLM_df_tmp.n50_SOIL3)-(CLM_df_tmp.n30_SOIL1+CLM_df_tmp.n30_SOIL2+CLM_df_tmp.n30_SOIL3)),2)
CLM_df.loc[:,"Nstock_min50_100"]    = round((CLM_df.TOTSOMN-(CLM_df_tmp.n50_SOIL1+CLM_df_tmp.n50_SOIL2+CLM_df_tmp.n50_SOIL3)),2)



Adjust below names to match the current relevant decomposition dataframes (if you have more than one):

In [ ]:
def add_stock_profile_columns(df):
    #Total (including litter pools)
    df.loc[:,"stock30"]      = round(df.c30_tot,2)
    df.loc[:,"stock30_50"]   = (df.c50_tot-df.c30_tot)
    df.loc[:,"stock50_100"]   = (df.TOTC-df.c50_tot)

    #Mineral soil only (excluding litter pools)
    df.loc[:,"stock_min30"] = (df.c30_SOM_tot)
    df.loc[:,"stock_min30_50"]    = (df.c50_SOM_tot-df.c30_SOM_tot)
    df.loc[:,"stock_min50_100"]    = (df.TOTSOMC-df.c50_SOM_tot)

def add_stock_profile_columns_nitrogen(df):
    #Total (including litter pools)
    df.loc[:,"Nstock30"]      = round(df.n30_tot,2)
    df.loc[:,"Nstock30_50"]   = (df.n50_tot-df.n30_tot)
    df.loc[:,"Nstock50_100"]   = (df.TOTN-df.n50_tot)

    #Mineral soil only (excluding litter pools)
    df.loc[:,"Nstock_min30"] = (df.n30_SOM_tot)
    df.loc[:,"Nstock_min30_50"]    = (df.n50_SOM_tot-df.n30_SOM_tot)
    df.loc[:,"Nstock_min50_100"]    = (df.TOTSOMN-df.n50_SOM_tot)


In [ ]:
add_stock_profile_columns(decomp_df)
add_stock_profile_columns_nitrogen(decomp_df)

# Add CUE to MIMICS+ dataframe

In [ ]:
# CUE = (total_microbial_uptake - microbial respiration)/total_microbial_uptake

CUE_list = []
for site in decomp_data:
    HR = (site.HRb+site.HRf+site.HRa+site.HRe)
    Input = (site.C_LITmSAPb+site.C_LITsSAPb+site.C_SOMaSAPb+site.C_LITmSAPf+site.C_LITsSAPf+site.C_SOMaSAPf+site.C_PlantAM+site.C_PlantEcM)
    HR_mass = (HR*depth[:len(HR.levsoi),None]).sum() #multply the input concentration in each layer with the layer thickness to get mass, then sum up the column with respect to both depth and time
    Input_mass = (Input*depth[:len(HR.levsoi),None]).sum() #gC/(m2

    CUE = 1-HR_mass/Input_mass
    CUE_list.append(float(CUE.values))
decomp_df["CUE"] = CUE_list

# Plots & analyses

### f02 (fig:total_profile) plot

In [ ]:
# Plot f02.pdf (total box plots)
sns.set_style("darkgrid")
all30     = [podz_wo_subset.stock30, obs_subset.stock30,decomp_df.stock30,CLM_df.stock30]
all30_50  = [podz_wo_subset[podz_wo_subset.stock30_50 > 0].stock30_50, obs_subset[obs_subset.stock30_50 > 0].stock30_50,decomp_df[obs_subset.stock30_50 > 0].stock30_50,CLM_df[obs_subset.stock30_50 > 0].stock30_50]
all50_100 = [podz_wo_subset[podz_wo_subset.stock50_100 > 0].stock50_100, obs_subset[obs_subset.stock50_100 > 0].stock50_100,decomp_df[obs_subset.stock50_100 > 0].stock50_100,CLM_df[obs_subset.stock50_100 > 0].stock50_100]
labels = ["OBS podzols","OBS subset","MIMICS+","CLM"]

fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(25, 6))

ax1.boxplot(all30,labels = labels,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax1.set_title(r"(a)",fontsize = 15, loc = "left")
ax1.set_title("0–30 cm",fontsize = 15, loc = "center")
ax1.set_ylabel("$\mathrm{kgC m^{-2}}$", fontsize=15);
ax1.tick_params(axis='both', labelsize=14);ax1.grid(True)
ax1.scatter(x=[1+random.uniform(-0.1, 0.1) for i in range(len(all30[0]))],y=all30[0], label = "N=%i"%len(all30[0]),alpha=0.5,color="gray")
ax1.scatter(x=[2+random.uniform(-0.1, 0.1) for i in range(len(all30[2]))],y=all30[1], label = "N=%i"%len(all30[1]),alpha=0.5,color=colors[0])
ax1.scatter(x=[3+random.uniform(-0.1, 0.1) for i in range(len(all30[1]))],y=all30[2], label = "N=%i"%len(all30[2]),alpha=0.5,color=colors[1])
ax1.scatter(x=[4+random.uniform(-0.1, 0.1) for i in range(len(all30[3]))],y=all30[3], label = "N=%i"%len(all30[3]),alpha=0.5,color=colors[2])
ax1.legend()

ax2.boxplot(all30_50,labels = labels,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax2.set_title(r"(b)",fontsize = 15, loc = "left")
ax2.set_title("30–50 cm",fontsize = 15, loc = "center")
ax2.tick_params(axis='both', labelsize=14)
ax2.scatter(x=[1+random.uniform(-0.1, 0.1) for i in range(len(all30_50[0]))],y=all30_50[0], label = "N=%i"%len(all30_50[0]),alpha=0.5,color="gray")
ax2.scatter(x=[2+random.uniform(-0.1, 0.1) for i in range(len(all30_50[1]))],y=all30_50[1], label = "N=%i"%len(all30_50[1]),alpha=0.5,color=colors[0])
ax2.scatter(x=[3+random.uniform(-0.1, 0.1) for i in range(len(all30_50[2]))],y=all30_50[2], label = "N=%i"%len(all30_50[2]),alpha=0.5,color=colors[1])
ax2.scatter(x=[4+random.uniform(-0.1, 0.1) for i in range(len(all30_50[3]))],y=all30_50[2], label = "N=%i"%len(all30_50[3]),alpha=0.5,color=colors[2])
ax2.legend()


ax3.boxplot(all50_100,labels = labels,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax3.set_title(r"(c)",fontsize = 15, loc = "left")
ax3.set_title("50–100 cm",fontsize = 15,loc="center")
ax3.tick_params(axis='both', labelsize=14);ax3.grid(True)
ax3.scatter(x=[1+random.uniform(-0.1, 0.1) for i in range(len(all50_100[0]))],y=all50_100[0], label = "N=%i"%len(all50_100[0]),alpha=0.5,color="gray")
ax3.scatter(x=[2+random.uniform(-0.1, 0.1)for i in range(len(all50_100[1]))],y=all50_100[1], label = "N=%i"%len(all50_100[1]),alpha=0.5,color=colors[0])
ax3.scatter(x=[3+random.uniform(-0.1, 0.1)for i in range(len(all50_100[2]))],y=all50_100[2], label = "N=%i"%len(all50_100[2]),alpha=0.5,color=colors[1])
ax3.scatter(x=[4+random.uniform(-0.1, 0.1)for i in range(len(all50_100[3]))],y=all50_100[3], label = "N=%i"%len(all50_100[3]),alpha=0.5,color=colors[2])
ax3.legend()

plt.savefig("./figures/f02.pdf",bbox_inches="tight")

### f02 analysis 

In [ ]:
def significance_ttest(data_series1,name1, data_series2,name2, alpha,equal_var):

    A = data_series1.values.tolist()
    B = data_series2.values.tolist()
    
    t_check = stats.ttest_ind(A, B,equal_var=equal_var)
    t_check
    if (t_check[1] < alpha):
        print('%s IS significantly different from %s with alpha = %1.3f' %(name1,name2,alpha))
    else:
        print('%s IS NOT significantly different from %s with alpha = %1.3f' %(name1,name2, alpha))
    print('t-statistic = %6.3f pvalue = %6.4f' % (t_check[0], t_check[1]))


print("--------------------0-30cm: -----------------------")
print("Equal variance = False --> Welch's test that does not assume equal variance: \n")
significance_ttest(obs_subset.stock30,"OBS 0-30cm",decomp_df.stock30,"MIMICS+ 0-30cm",alpha = 0.05,equal_var = False)
significance_ttest(obs_subset.stock30,"OBS 0-30cm",CLM_df.stock30,"CLM 0-30cm",alpha = 0.05,equal_var = False)
significance_ttest(decomp_df.stock30,"MIMICS+ 0-30cm",CLM_df.stock30,"CLM 0-30cm",alpha = 0.05,equal_var = False)

significance_ttest(podz_wo_subset.stock30,"podzols 0-30cm",obs_subset.stock30,"OBS 0-30cm",alpha = 0.05,equal_var = False)
significance_ttest(podz_wo_subset.stock30,"podzols 0-30cm",decomp_df.stock30,"MIMICS+ 0-30cm",alpha = 0.05,equal_var = False)
significance_ttest(podz_wo_subset.stock30,"podzols 0-30cm",CLM_df.stock30,"CLM 0-30cm",alpha = 0.05,equal_var = False)


print("\n---------------------30-50 cm: -----------------------")
print("Equal variance = False --> Welch's test that does not assume equal variance: \n")
significance_ttest(obs_subset[obs_subset.stock30_50 > 0].stock30_50,"OBS 30-50cm",decomp_df[obs_subset.stock30_50 > 0].stock30_50,"MIMICS+ 30-50cm",alpha = 0.05,equal_var = False)
significance_ttest(obs_subset[obs_subset.stock30_50 > 0].stock30_50,"OBS 30-50cm",CLM_df[obs_subset.stock30_50 > 0].stock30_50,"CLM 30-50cm",alpha = 0.05,equal_var = False)
significance_ttest(decomp_df[obs_subset.stock30_50 > 0].stock30_50,"MIMICS+ 30-50cm",CLM_df[obs_subset.stock30_50 > 0].stock30_50,"CLM 30-50cm",alpha = 0.05,equal_var = False)

significance_ttest(podz_wo_subset[podz_wo_subset.stock30_50 > 0].stock30_50,"podzols 30-50cm",obs_subset[obs_subset.stock30_50 > 0].stock30_50,"OBS 30-50cm",alpha = 0.05,equal_var = False)
significance_ttest(podz_wo_subset[podz_wo_subset.stock30_50 > 0].stock30_50,"podzols 30-50cm",decomp_df[obs_subset.stock30_50 > 0].stock30_50,"MIMICS+ 30-50cm",alpha = 0.05,equal_var = False)
significance_ttest(podz_wo_subset[podz_wo_subset.stock30_50 > 0].stock30_50,"podzols 30-50cm",CLM_df[obs_subset.stock30_50 > 0].stock30_50,"CLM 30-50cm",alpha = 0.05,equal_var = False)


print("\n---------------------50-100 cm: -----------------------")
print("Equal variance = False --> Welch's test that does not assume equal variance: \n")
significance_ttest(obs_subset[obs_subset.stock50_100 > 0].stock50_100,"OBS 50-100cm",decomp_df[obs_subset.stock50_100 > 0].stock50_100,"MIMICS+ 50-100cm",alpha = 0.05,equal_var = False)
significance_ttest(obs_subset[obs_subset.stock50_100 > 0].stock50_100,"OBS 50-100cm",CLM_df[obs_subset.stock50_100 > 0].stock50_100,"CLM 50-100cm",alpha = 0.05,equal_var = False)
significance_ttest(decomp_df[obs_subset.stock50_100 > 0].stock50_100,"MIMICS+ 50-100cm",CLM_df[obs_subset.stock50_100 > 0].stock50_100,"CLM 50-100cm",alpha = 0.05,equal_var = False)

significance_ttest(podz_wo_subset[podz_wo_subset.stock50_100 > 0].stock50_100,"podzols 50-100cm",obs_subset[obs_subset.stock50_100 > 0].stock50_100,"OBS 50-100cm",alpha = 0.05,equal_var = False)
significance_ttest(podz_wo_subset[podz_wo_subset.stock50_100 > 0].stock50_100,"podzols 50-100cm",decomp_df[obs_subset.stock50_100 > 0].stock50_100,"MIMICS+ 50-100cm",alpha = 0.05,equal_var = False)
significance_ttest(podz_wo_subset[podz_wo_subset.stock50_100 > 0].stock50_100,"podzols 50-100cm",CLM_df[obs_subset.stock50_100 > 0].stock50_100,"CLM 50-100cm",alpha = 0.05,equal_var = False)


### f03 (fig:CNboxplot) plot

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(nrows=1,ncols=4,figsize=[30,6])

ax1.boxplot([podz_wo_subset.cstock/podz_wo_subset.nstock,obs_subset.cstock/obs_subset.nstock,decomp_df.TOTC/decomp_df.TOTN,CLM_df.TOTC/CLM_df.TOTN], labels = ["OBS podzols","OBS subset","MIMICS+","CLM"],vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax1.set_title("(a)",fontsize=15,loc = "left")
ax1.set_title("Total soil",fontsize=15,loc="center")
ax1.set_ylabel("C:N ratio", fontsize=15);ax1.set_ylim(8,76)
ax1.tick_params(axis='both', labelsize=14)

ax2.boxplot([(podz_wo_subset.c100min/podz_wo_subset.n100min).dropna(),obs_subset.c100min/obs_subset.n100min,decomp_df.TOTSOMC/decomp_df.TOTSOMN, CLM_df.TOTSOMC/CLM_df.TOTSOMN],labels = ["OBS podzols\nmineral soil","OBS\nmineral soil", "MIMICS+\nSOM", "CLM\nSOM"],vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax2.set_title("(b)",fontsize=15,loc = "left")
ax2.set_title("Mineral soil/SOM",fontsize=15,loc="center")
ax2.set_ylim(8,76)
ax2.tick_params(axis='both', labelsize=14)

ax3.boxplot([podz_wo_subset.CNFOREST.dropna(),obs_subset.CNFOREST,decomp_df.TOTLITC/decomp_df.TOTLITN,CLM_df.TOTLITC/CLM_df.TOTLITN],labels = ["OBS podzols\nLFH","OBS\nLFH","MIMICS+\nlitter","CLM\nlitter"],vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax3.set_title("(c)",fontsize=15,loc="left");
ax3.set_title("LFH layer/litter",fontsize=15,loc="center");
ax3.tick_params(axis='both', labelsize=14)

ax4.boxplot([decomp_df.stock30/decomp_df.Nstock30, decomp_df.stock30_50/decomp_df.Nstock30_50, decomp_df.stock50_100/decomp_df.Nstock50_100],
            labels = ["0–30cm", "30–50cm", "50–100cm"],vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax4.set_title("(d)",fontsize=15,loc="left");ax4.set_ylim(8,76)
ax4.set_title("MIMICS+ (all pools)",fontsize=15,loc="center");
ax4.tick_params(axis='both', labelsize=14)


plt.savefig("./figures/f03_include_podzols.pdf",bbox_inches="tight")

In [ ]:
positions1=[.5,2.5,4.5]
positions2=[1.1,3.1,5.1]
sns.set_style("darkgrid")
fig,(ax1,ax2,ax3,ax4) = plt.subplots(nrows=1,ncols=4,figsize=[30,6])

ax1.boxplot([obs_subset.cstock/obs_subset.nstock,decomp_df.TOTC/decomp_df.TOTN,CLM_df.TOTC/CLM_df.TOTN],labels = ["OBS","MIMICS+","CLM"],vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax1.set_title("(a)",fontsize=15,loc = "left")
ax1.set_title("Total soil",fontsize=15,loc="center")
ax1.set_ylabel("C:N ratio", fontsize=15);ax1.set_ylim(8,76)
ax1.tick_params(axis='both', labelsize=14)

ax2.boxplot([obs_subset.c100min/obs_subset.n100min,decomp_df.TOTSOMC/decomp_df.TOTSOMN, CLM_df.TOTSOMC/CLM_df.TOTSOMN],labels = ["OBS\nmineral soil", "MIMICS+\nSOM", "CLM\nSOM"],vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax2.set_title("(b)",fontsize=15,loc = "left")
ax2.set_title("Mineral soil/SOM",fontsize=15,loc="center")
ax2.set_ylim(8,76)
ax2.tick_params(axis='both', labelsize=14)

ax3.boxplot([obs_subset.CNFOREST,decomp_df.TOTLITC/decomp_df.TOTLITN,CLM_df.TOTLITC/CLM_df.TOTLITN],labels = ["OBS\nLFH","MIMICS+\nlitter","CLM\nlitter"],vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax3.set_title("(c)",fontsize=15,loc="left");
ax3.set_title("LFH layer/litter",fontsize=15,loc="center");
ax3.tick_params(axis='both', labelsize=14)

ax4.boxplot([decomp_df.stock30/decomp_df.Nstock30, decomp_df.stock30_50/decomp_df.Nstock30_50, decomp_df.stock50_100/decomp_df.Nstock50_100],
            labels = ["0–30cm", "30–50cm", "50–100cm"],vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"});
ax4.set_title("(d)",fontsize=15,loc="left");ax4.set_ylim(8,76)
ax4.set_title("MIMICS+ (all pools)",fontsize=15,loc="center");
ax4.tick_params(axis='both', labelsize=14)


plt.savefig("./figures/f03.pdf",bbox_inches="tight")

### f03 analysis

In [ ]:
print("---SIGNIFICANCE TTESTS for CN ratios MIMICS+ vertical:  ")

significance_ttest(decomp_df.stock30/decomp_df.Nstock30,"MIMICS 0-30cm",decomp_df.stock30_50/decomp_df.Nstock30_50,"MIMICS+ 30-50cm",alpha=0.05,equal_var=False )
significance_ttest(decomp_df.stock30/decomp_df.Nstock30,"MIMICS 0-30cm",decomp_df.stock50_100/decomp_df.Nstock50_100,"MIMICS+ 50-100cm",alpha=0.05,equal_var=False )
significance_ttest(decomp_df.stock30_50/decomp_df.Nstock30_50,"MIMICS 30-50cm",decomp_df.stock50_100/decomp_df.Nstock50_100,"MIMICS+ 50-100cm",alpha=0.05,equal_var=False )

print("\n0-30cm: ")
print((decomp_df.stock30/decomp_df.Nstock30).describe())
print("30-50cm")
print((decomp_df.stock30_50/decomp_df.Nstock30_50).describe())
print("50-100cm: ")
print((decomp_df.stock50_100/decomp_df.Nstock50_100).describe())

In [ ]:
print("---SIGNIFICANCE TTESTS for CN ratios: ")
mf.significance_ttest(obs_subset.cstock/obs_subset.nstock,"OBS TOT CN",decomp_df.TOTC/decomp_df.TOTN,"MIMICS+ TOTCN",alpha=0.05,equal_var=False )
mf.significance_ttest(obs_subset.cstock/obs_subset.nstock,"OBS TOT CN",CLM_df.TOTC/CLM_df.TOTN,"CLM TOTCN",alpha=0.05,equal_var=False )
mf.significance_ttest(decomp_df.TOTC/decomp_df.TOTN,"MIMICS+ TOTCN",CLM_df.TOTC/CLM_df.TOTN,"CLM TOTCN",alpha=0.05,equal_var=False )
print("-----")
mf.significance_ttest(podz_wo_subset.cstock/podz_wo_subset.nstock,"OBS podzols",obs_subset.cstock/obs_subset.nstock,"OBS subset CN",alpha=0.05,equal_var=False )
mf.significance_ttest(podz_wo_subset.cstock/podz_wo_subset.nstock,"OBS podzols",decomp_df.TOTC/decomp_df.TOTN,"MIMICS+ TOTCN",alpha=0.05,equal_var=False )
mf.significance_ttest(podz_wo_subset.cstock/podz_wo_subset.nstock,"OBS podzols",CLM_df.TOTC/CLM_df.TOTN,"CLM TOTCN",alpha=0.05,equal_var=False )

print("\nOBS podzols: ")
print((podz_wo_subset.cstock/podz_wo_subset.nstock).describe())
print("\nOBS subset: ")
print((obs_subset.cstock/obs_subset.nstock).describe())
print("MIMICS+: ")
print((decomp_df.TOTC/decomp_df.TOTN).describe())
print("CLM: ")
print((CLM_df.TOTC/CLM_df.TOTN).describe())

In [ ]:
print("---SIGNIFICANCE TTESTS for CN ratios in mineral soil: ")
mf.significance_ttest(obs_subset.c100min/obs_subset.n100min,"OBS mineral CN",decomp_df.TOTSOMC/decomp_df.TOTSOMN,"MIMICS+ SOM CN",alpha=0.05,equal_var=False )
mf.significance_ttest(obs_subset.c100min/obs_subset.c100min,"OBS mineral soil CN",CLM_df.TOTSOMC/CLM_df.TOTSOMN,"CLM TOTSOMCN",alpha=0.05,equal_var=False )
mf.significance_ttest(decomp_df.TOTSOMC/decomp_df.TOTSOMN,"MIMICS+ TOTSOMCN",CLM_df.TOTSOMC/CLM_df.TOTSOMN,"CLM TOTSOMCN",alpha=0.05,equal_var=False )
print("-----")
mf.significance_ttest((podz_wo_subset.c100min/podz_wo_subset.n100min).notna(),"OBS podzols",obs_subset.c100min/obs_subset.n100min,"OBS mineral soil CN",alpha=0.05,equal_var=False )
mf.significance_ttest((podz_wo_subset.c100min/podz_wo_subset.n100min).notna(),"OBS podzols",decomp_df.TOTSOMC/decomp_df.TOTSOMN,"MIMICS+ TOTSOMCN",alpha=0.05,equal_var=False )
mf.significance_ttest((podz_wo_subset.c100min/podz_wo_subset.n100min).notna(),"OBS podzols",CLM_df.TOTSOMC/CLM_df.TOTSOMN,"CLM TOTSOMCN",alpha=0.05,equal_var=False )

print("\nOBS podzols: ")
print((podz_wo_subset.c100min/podz_wo_subset.n100min).describe())
print("\nOBS: ")
print((obs_subset.c100min/obs_subset.n100min).describe())
print("MIMICS+: ")
print((decomp_df.TOTSOMC/decomp_df.TOTSOMN).describe())
print("CLM: ")
print((CLM_df.TOTSOMC/CLM_df.TOTSOMN).describe())

In [ ]:
print("---SIGNIFICANCE TTESTS for LFH/litter  ratios: ")
mf.significance_ttest(obs_subset.CNFOREST,"OBS LFH CN",decomp_df.TOTLITC/decomp_df.TOTLITN,"MIMICS+ LITCN",alpha=0.05,equal_var=False )
mf.significance_ttest(obs_subset.CNFOREST,"OBS LFH CN",CLM_df.TOTLITC/CLM_df.TOTLITN,"CLM LITCN",alpha=0.05,equal_var=False )
mf.significance_ttest(decomp_df.TOTLITC/decomp_df.TOTLITN,"MIMICS+ LITCN",CLM_df.TOTLITC/CLM_df.TOTLITN,"CLM LITCN",alpha=0.05,equal_var=False )
print("-----")
mf.significance_ttest(podz_wo_subset.CNFOREST.notna(),"OBS podzols",obs_subset.CNFOREST,"OBS subset CNFOREST",alpha=0.05,equal_var=False )
mf.significance_ttest(podz_wo_subset.CNFOREST.notna(),"OBS podzols",decomp_df.TOTLITC/decomp_df.TOTLITN,"MIMICS+ LITCN",alpha=0.05,equal_var=False )
mf.significance_ttest(podz_wo_subset.CNFOREST.notna(),"OBS podzols",CLM_df.TOTC/CLM_df.TOTN,"CLM TOTCN",alpha=0.05,equal_var=False )

print("\nOBS podzols: ")
print((podz_wo_subset.CNFOREST).describe())
print("\nOBS: ")
print((obs_subset.CNFOREST).describe())
print("MIMICS+: ")
print((decomp_df.TOTLITC/decomp_df.TOTLITN).describe())
print("CLM: ")
print((CLM_df.TOTLITC/CLM_df.TOTLITN).describe())

### f04 (fig:boxplot_fraction) plot

In [ ]:
sns.set_style("darkgrid")
# Boxplots of fraction of SOC pools:
label = ["SOM","Litter","Microbes"]
fig,(ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=[10,5],sharey=True)
ax1.boxplot(x = [((decomp_df.SOMp + decomp_df.SOMc+decomp_df.SOMa)/1000)/decomp_df.TOTC,
                  ((decomp_df.LITm+decomp_df.LITs)/1000)/decomp_df.TOTC,            
                (decomp_df.MYC+decomp_df.SAP)/decomp_df.TOTC],
                labels = label,showmeans=True,medianprops={"color":"black"},
                meanprops={"marker":"D","markerfacecolor":"black","markeredgecolor":"black"}, whis=[0,100],
                widths=0.5);
ax1.set_ylabel("Fraction [-]",fontsize=15);
ax1.set_title("(a)",loc="left",fontsize=15);


tot_mic = decomp_df.SAPb+decomp_df.SAPf+decomp_df.EcM+decomp_df.AM
tot_LIT = decomp_df.LITm + decomp_df.LITs
tot_som = decomp_df.SOMp+decomp_df.SOMa+decomp_df.SOMc

a_frac = decomp_df.SOMa/tot_som
c_frac = decomp_df.SOMc/tot_som
p_frac = decomp_df.SOMp/tot_som

am_frac = decomp_df.AM/tot_mic
ecm_frac = decomp_df.EcM/tot_mic
f_frac = decomp_df.SAPf/tot_mic
b_frac = decomp_df.SAPb/tot_mic

m_frac = decomp_df.LITm/tot_LIT
s_frac = decomp_df.LITs/tot_LIT

width=0.4
ecolor="black"
p=ax2.bar(label[0],p_frac.mean(),width=width,color="darkgray", edgecolor=ecolor)
c=ax2.bar(label[0],c_frac.mean(),bottom=p_frac.mean(),width=width,color="lightgray", edgecolor=ecolor)
a=ax2.bar(label[0],a_frac.mean(),bottom=p_frac.mean()+c_frac.mean(),width=width,color="whitesmoke", edgecolor=ecolor)

s=ax2.bar(label[1],s_frac.mean(),width=width,color="darkgray", edgecolor=ecolor)
m=ax2.bar(label[1],m_frac.mean(),bottom=s_frac.mean(),width=width,color="lightgray", edgecolor=ecolor)

f=ax2.bar(label[2],f_frac.mean(),width=width,color="darkgray", edgecolor=ecolor)
b=ax2.bar(label[2],b_frac.mean(),bottom=f_frac.mean(),width=width,color="lightgray", edgecolor=ecolor)
my=ax2.bar(label[2],ecm_frac.mean()+am_frac.mean(),bottom=f_frac.mean()+b_frac.mean(),width=width,color="whitesmoke", edgecolor=ecolor)

ax2.bar_label(p, fmt='SOMp\n%1.2f'%p_frac.mean(),label_type="center")
ax2.bar_label(c, fmt='SOMc\n%1.2f'%c_frac.mean(),label_type="center")
ax2.bar_label(a, fmt='SOMa\n%1.2f'%a_frac.mean(),label_type="center")
ax2.bar_label(s, fmt='LITs\n%1.2f'%s_frac.mean(),label_type="center")
ax2.bar_label(m, fmt='LITm\n%1.2f'%m_frac.mean(),label_type="center")
ax2.bar_label(f, fmt='SAPf\n%1.2f'%f_frac.mean(),label_type="center")
ax2.bar_label(b, fmt='SAPb\n%1.2f'%b_frac.mean(),label_type="center")
ax2.bar_label(my, fmt='MYC %1.2f'%(ecm_frac.mean()+am_frac.mean()),label_type="edge")
ax2.set_title("(b)",loc="left",fontsize=15);
ax2.set_ylim(0,1.07);

#plt.savefig("./figures/f04.pdf",bbox_inches="tight")


### f05 (fig:heatmap) plot

In [ ]:
def plot_correlation_matrix(dataframe, cut_off, method,save_bol=False,save_name=None):
    #https://stackoverflow.com/questions/25571882/pandas-columns-correlation-with-statistical-significance
    df = dataframe.corr(method=method)
    mask = np.triu(np.ones_like(df, dtype=bool))
    mask |= np.abs(df) < cut_off
    dataframe_copy = df[~mask]  # fill in NaN in the non-desired cells
    nr_of_vars = len(dataframe_copy)
  
    if method == "spearman":
        pval = dataframe.corr(method=lambda x, y: spearmanr(x, y)[
                              1]) - np.eye(*dataframe_copy.shape)
        title = "Spearman correlation heatmap"
  
    elif method == "pearson":
        pval = dataframe.corr(method=lambda x, y: pearsonr(x, y)[
                              1]) - np.eye(*dataframe_copy.shape)

    else:
        print("Invalid method")

    p = pval.applymap(lambda x: ''.join(
        ['*' for t in [.05, .01, .001] if x <= t]))

    with sns.axes_style("dark"):
        f, ax = plt.subplots(figsize=(0.9*nr_of_vars, 0.7*nr_of_vars))
        ax = sns.heatmap(dataframe_copy, vmin=-1, vmax=1, annot=dataframe_copy.round(2).astype(
            str) + p, fmt='', annot_kws={"size": 10}, mask=mask, linewidth=.3, cmap="coolwarm")
        plt.yticks(rotation=0)
    ax.set_title(title, fontsize = 15)
    textstr = '\n '.join((
            r'Significance level:',
            r'***   : $\alpha   =   0.001$ ',
            r'**    : $\alpha   =   0.01$ ',
            r'*      : $\alpha   =   0.05$ '))

    # these are matplotlib.patch.Patch properties
    props = dict( facecolor='white', alpha=0.9)

    # place a text box in upper left in axes coords
    ax.text(0.76, 0.99, textstr, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=props,multialignment="left")
    if save_bol:
        f.savefig("./figures/%s.pdf"%save_name,bbox_inches="tight")

    

In [ ]:
plot_correlation_matrix(decomp_df.rename(columns={"TOTC_input":"C_input","HR_flux":"HR"})[["TOTC","C_input","HR","CUE","pct_microbes","FBratio","SOILWATER","MAP","MAT","NH4_sol","NH4_sorp","NO3"]],cut_off = 0.0,method="spearman",save_bol=False,save_name="f05")

#### fig:obs_heatmap

In [ ]:
plot_correlation_matrix(obs_subset.rename(columns={"cstock":"TOTC","nstock":"TOTN"})[["TOTC","CLFH","C30","C50","TOTN","NLFH","MAT","MAP","KOORDINA","ALTITUDE"]],cut_off = 0.0,method="spearman",save_bol=True,save_name="obs_heatmap")

In [ ]:
plot_correlation_matrix(podz_wo_subset.rename(columns={"cstock":"TOTC","nstock":"TOTN"})[["TOTC","CLFH","C30","C50","TOTN","NLFH","MAT","MAP","KOORDINA","ALTITUDE"]],cut_off = 0.0,method="spearman",save_bol=True,save_name="podzol_heatmap")

#### fig:CLM_heatmap

In [ ]:
plot_correlation_matrix(CLM_df[['TSA', 'TOTSOMC', 'TOTSOMN', 'TOTLITC','TOTLITN', 'MAP', 'HR', 'TOTN', 'TOTC', 'MAT', 'SOILWATER']],cut_off = 0.0,method="spearman",save_bol=True,save_name="CLM_heatmap")

### f06 (fig:clim_profile) plot

In [ ]:
#Distributions where sites are removed from OBS distributions AND modeled distributions if stock is zero for OBS

#Wetter and drier: 
w30 = [obs_subset.query("MAP_cat=='wetter'").stock30,decomp_df.query("MAP_cat=='wetter'").stock30,CLM_df.query("MAP_cat=='wetter'").stock30]
d30 = [obs_subset.query("MAP_cat=='drier'").stock30,decomp_df.query("MAP_cat=='drier'").stock30,CLM_df.query("MAP_cat=='drier'").stock30]

w30_50 = [obs_subset.query("MAP_cat=='wetter'")[obs_subset.stock30_50 > 0].stock30_50,decomp_df.query("MAP_cat=='wetter'")[obs_subset.stock30_50 > 0].stock30_50,CLM_df.query("MAP_cat=='wetter'")[obs_subset.stock30_50 > 0].stock30_50]

d30_50 = [ obs_subset.query("MAP_cat=='drier'")[obs_subset.stock30_50 > 0].stock30_50,decomp_df.query("MAP_cat=='drier'")[obs_subset.stock30_50 > 0].stock30_50,CLM_df.query("MAP_cat=='drier'")[obs_subset.stock30_50 > 0].stock30_50]

w50_100 = [ obs_subset.query("MAP_cat=='wetter'")[obs_subset.stock50_100 > 0].stock50_100, decomp_df.query("MAP_cat=='wetter'")[obs_subset.stock50_100 > 0].stock50_100,CLM_df.query("MAP_cat=='wetter'")[obs_subset.stock50_100 > 0].stock50_100]
d50_100 = [ obs_subset.query("MAP_cat=='drier'")[obs_subset.stock50_100 > 0].stock50_100, decomp_df.query("MAP_cat=='drier'")[obs_subset.stock50_100 > 0].stock50_100, CLM_df.query("MAP_cat=='drier'")[obs_subset.stock50_100 > 0].stock50_100]

#warmer and cooler
c30 = [ obs_subset.query("MAT_cat=='cooler'").stock30,decomp_df.query("MAT_cat=='cooler'").stock30,CLM_df.query("MAT_cat=='cooler'").stock30]
wa30 = [ obs_subset.query("MAT_cat=='warmer'").stock30,decomp_df.query("MAT_cat=='warmer'").stock30,CLM_df.query("MAT_cat=='warmer'").stock30]
        
c30_50 = [obs_subset.query("MAT_cat=='cooler'")[obs_subset.stock30_50 > 0].stock30_50,decomp_df.query("MAT_cat=='cooler'")[obs_subset.stock30_50 > 0].stock30_50,CLM_df.query("MAT_cat=='cooler'")[obs_subset.stock30_50 > 0].stock30_50]
wa30_50 = [ obs_subset.query("MAT_cat=='warmer'")[obs_subset.stock30_50 > 0].stock30_50, decomp_df.query("MAT_cat=='warmer'")[obs_subset.stock30_50 > 0].stock30_50, CLM_df.query("MAT_cat=='warmer'")[obs_subset.stock30_50 > 0].stock30_50]

c50_100 = [ obs_subset.query("MAT_cat=='cooler'")[obs_subset.stock50_100 > 0].stock50_100, decomp_df.query("MAT_cat=='cooler'")[obs_subset.stock50_100 > 0].stock50_100, CLM_df.query("MAT_cat=='cooler'")[obs_subset.stock50_100 > 0].stock50_100]
wa50_100 = [  obs_subset.query("MAT_cat=='warmer'")[obs_subset.stock50_100 > 0].stock50_100,  decomp_df.query("MAT_cat=='warmer'")[obs_subset.stock50_100 > 0].stock50_100,  CLM_df.query("MAT_cat=='warmer'")[obs_subset.stock50_100 > 0].stock50_100]

font_dict={"fontsize":"x-large","fontweight":"bold"}
labels_1stline_MAP=["drier  wetter","drier  wetter","drier  wetter"]
labels_1stline_MAT=["cooler  warmer","cooler  warmer","cooler  warmer"]
labels_2ndline=["\nOBS","\nMIMICS+","\nCLM"]
positions1=[.5,2.5,4.5]
positions2=[1.1,3.1,5.1]
ticks_pos=[0.8,2.8,4.8,0.8,2.8,4.8]

fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(nrows=2, ncols=3, figsize=(21, 14))

ax1.boxplot(c30,positions=positions1,labels = labels_1stline_MAT,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[0],"markeredgecolor":"black"});
ax1.boxplot(wa30,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[1],"markeredgecolor":"black"});
ax1.set_title("(a)",fontsize = 15, loc = "left");
ax1.set_title("0–30 cm",fontsize = 15, loc = "center");
ax1.set_ylabel("$\mathrm{kgC m^{-2}}$",fontsize=15);
ax1.set_xticks(ticks_pos);
ax1.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(c30[0]))],y=c30[0], label = "N=%i"%len(c30[0]),color=colors[0],alpha=0.5)
ax1.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(wa30[0]))],y=wa30[0], label = "N=%i"%len(wa30[0]),color=colors[0],alpha=0.5,marker='^')

ax1.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(c30[1]))],y=c30[1], label = "N=%i"%len(c30[1]),color=colors[1],alpha=0.5)
ax1.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(wa30[1]))],y=wa30[1], label = "N=%i"%len(wa30[1]),color=colors[1],alpha=0.5,marker='^')

ax1.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(c30[2]))],y=c30[2], label = "N=%i"%len(c30[2]),color=colors[2],alpha=0.5)
ax1.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(wa30[2]))],y=wa30[2], label = "N=%i"%len(wa30[2]),color=colors[2],alpha=0.5,marker='^')
ax1.legend()

ax2.boxplot(c30_50,positions=positions1,labels = labels_1stline_MAT,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[0],"markeredgecolor":"black"});
ax2.boxplot(wa30_50,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[1],"markeredgecolor":"black"});
ax2.set_title("(b)", fontsize = 15, loc = "left")
ax2.set_title(" 30–50 cm ", fontsize = 15, loc = "center")
ax2.set_xticks(ticks_pos);
ax2.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(c30_50[0]))],y=c30_50[0], label = "N=%i"%len(c30_50[0]),color=colors[0],alpha=0.5)
ax2.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(wa30_50[0]))],y=wa30_50[0], label = "N=%i"%len(wa30_50[0]),color=colors[0],alpha=0.5,marker='^')
ax2.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(c30_50[1]))],y=c30_50[1], label = "N=%i"%len(c30_50[1]),color=colors[1],alpha=0.5)
ax2.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(wa30_50[1]))],y=wa30_50[1], label = "N=%i"%len(wa30_50[1]),color=colors[1],alpha=0.5,marker='^')
ax2.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(c30_50[2]))],y=c30_50[2], label = "N=%i"%len(c30_50[2]),color=colors[2],alpha=0.5)
ax2.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(wa30_50[2]))],y=wa30_50[2], label = "N=%i"%len(wa30_50[2]),color=colors[2],alpha=0.5,marker='^')
ax2.legend()

ax3.boxplot(c50_100,positions=positions1,vert=True,showmeans=True,labels = labels_1stline_MAT,medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[0],"markeredgecolor":"black"});
ax3.boxplot(wa50_100,positions=positions2,vert=True,showmeans=True,labels = labels_2ndline,medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[1],"markeredgecolor":"black"});
ax3.set_title("(c)",fontsize = 15, loc = "left");
ax3.set_title("50–100 cm",fontsize = 15, loc = "center");
ax3.set_xticks(ticks_pos);
ax3.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(c50_100[0]))],y=c50_100[0], label = "N=%i"%len(c50_100[0]),color=colors[0],alpha=0.5)
ax3.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(wa50_100[0]))],y=wa50_100[0], label = "N=%i"%len(wa50_100[0]),color=colors[0],alpha=0.5,marker='^')
ax3.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(c50_100[1]))],y=c50_100[1], label = "N=%i"%len(c50_100[1]),color=colors[1],alpha=0.5)
ax3.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(wa50_100[1]))],y=wa50_100[1], label = "N=%i"%len(wa50_100[1]),color=colors[1],alpha=0.5,marker='^')
ax3.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(c50_100[2]))],y=c50_100[2], label = "N=%i"%len(c50_100[2]),color=colors[2],alpha=0.5)
ax3.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(wa50_100[2]))],y=wa50_100[2], label = "N=%i"%len(wa50_100[2]),color=colors[2],alpha=0.5,marker='^')
ax3.legend()

ax4.boxplot(d30,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"});
ax4.boxplot(w30,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"});
ax4.set_title("(d)",fontsize = 15, loc = "left");
ax4.set_title("0–30 cm",fontsize = 15, loc = "center");
ax4.set_ylabel("$\mathrm{kgC m^{-2}}$",fontsize=16);
ax4.set_xticks(ticks_pos);
ax4.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(d30[0]))],y=d30[0], label = "N=%i"%len(d30[0]),color=colors[0],alpha=0.5)
ax4.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(w30[0]))],y=w30[0], label = "N=%i"%len(w30[0]),color=colors[0],alpha=0.5,marker='^')
ax4.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(d30[1]))],y=d30[1], label = "N=%i"%len(d30[1]),color=colors[1],alpha=0.5)
ax4.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(w30[1]))],y=w30[1], label = "N=%i"%len(w30[1]),color=colors[1],alpha=0.5,marker='^')
ax4.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(d30[2]))],y=d30[2], label = "N=%i"%len(d30[2]),color=colors[2],alpha=0.5)
ax4.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(w30[2]))],y=w30[2], label = "N=%i"%len(w30[2]),color=colors[2],alpha=0.5,marker='^')
ax4.legend()

ax5.boxplot(d30_50,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"});
ax5.boxplot(w30_50,positions=positions2,labels =labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"});
ax5.set_title("(e)",fontsize = 15, loc = "left");
ax5.set_title("30–50 cm",fontsize = 15, loc = "center");
ax5.set_xticks(ticks_pos);
ax5.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(d30_50[0]))],y=d30_50[0], label = "N=%i"%len(d30_50[0]),color=colors[0],alpha=0.5)
ax5.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(w30_50[0]))],y=w30_50[0], label = "N=%i"%len(w30_50[0]),color=colors[0],alpha=0.5,marker='^')
ax5.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(d30_50[1]))],y=d30_50[1], label = "N=%i"%len(d30_50[1]),color=colors[1],alpha=0.5)
ax5.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(w30_50[1]))],y=w30_50[1], label = "N=%i"%len(w30_50[1]),color=colors[1],alpha=0.5,marker='^')
ax5.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(d30_50[2]))],y=d30_50[2], label = "N=%i"%len(d30_50[2]),color=colors[2],alpha=0.5)
ax5.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(w30_50[2]))],y=w30_50[2], label = "N=%i"%len(w30_50[2]),color=colors[2],alpha=0.5,marker='^')
ax5.legend()

ax6.boxplot(d50_100,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"});
ax6.boxplot(w50_100,positions=positions2,labels =labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"});
ax6.set_title(r"$c)$"+"\t\t\t50-100 cm",fontsize = 15, loc = "left");
ax6.set_title(r"(f)",fontsize = 15, loc = "left");
ax6.set_title("50–100 cm",fontsize = 15, loc = "center");
ax6.set_xticks(ticks_pos);
ax6.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(d50_100[0]))],y=d50_100[0], label = "N=%i"%len(d50_100[0]),color=colors[0],alpha=0.5)
ax6.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(w50_100[0]))],y=w50_100[0], label = "N=%i"%len(w50_100[0]),color=colors[0],alpha=0.5,marker='^')
ax6.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(d50_100[1]))],y=d50_100[1], label = "N=%i"%len(d50_100[1]),color=colors[1],alpha=0.5)
ax6.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(w50_100[1]))],y=w50_100[1], label = "N=%i"%len(w50_100[1]),color=colors[1],alpha=0.5,marker='^')
ax6.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(d50_100[2]))],y=d50_100[2], label = "N=%i"%len(d50_100[2]),color=colors[2],alpha=0.5)
ax6.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(w50_100[2]))],y=w50_100[2], label = "N=%i"%len(w50_100[2]),color=colors[2],alpha=0.5,marker='^')
ax6.legend()


#plt.savefig("./figures/f06.pdf",bbox_inches="tight")


In [ ]:
#Distributions where sites are removed from OBS distributions AND modeled distributions if stock is zero for OBS

#Wetter and drier: 
w30 = [podz_wo_subset.query("MAP_cat=='wetter'")[podz_wo_subset.stock30 > 0].stock30, obs_subset.query("MAP_cat=='wetter'").stock30,decomp_df.query("MAP_cat=='wetter'").stock30,CLM_df.query("MAP_cat=='wetter'").stock30]
d30 = [podz_wo_subset.query("MAP_cat=='drier'")[podz_wo_subset.stock30 > 0].stock30, obs_subset.query("MAP_cat=='drier'").stock30,decomp_df.query("MAP_cat=='drier'").stock30,CLM_df.query("MAP_cat=='drier'").stock30]

w30_50 = [podz_wo_subset.query("MAP_cat=='wetter'")[podz_wo_subset.stock30_50 > 0].stock30_50,
          obs_subset.query("MAP_cat=='wetter'")[obs_subset.stock30_50 > 0].stock30_50,
          decomp_df.query("MAP_cat=='wetter'")[obs_subset.stock30_50 > 0].stock30_50,
          CLM_df.query("MAP_cat=='wetter'")[obs_subset.stock30_50 > 0].stock30_50]

d30_50 = [podz_wo_subset.query("MAP_cat=='drier'")[podz_wo_subset.stock30_50 > 0].stock30_50, 
          obs_subset.query("MAP_cat=='drier'")[obs_subset.stock30_50 > 0].stock30_50,
          decomp_df.query("MAP_cat=='drier'")[obs_subset.stock30_50 > 0].stock30_50,
          CLM_df.query("MAP_cat=='drier'")[obs_subset.stock30_50 > 0].stock30_50]

w50_100 = [podz_wo_subset.query("MAP_cat=='wetter'")[podz_wo_subset.stock50_100 > 0].stock50_100, 
           obs_subset.query("MAP_cat=='wetter'")[obs_subset.stock50_100 > 0].stock50_100,
           decomp_df.query("MAP_cat=='wetter'")[obs_subset.stock50_100 > 0].stock50_100,
           CLM_df.query("MAP_cat=='wetter'")[obs_subset.stock50_100 > 0].stock50_100]
d50_100 = [podz_wo_subset.query("MAP_cat=='drier'")[podz_wo_subset.stock50_100 > 0].stock50_100,
           obs_subset.query("MAP_cat=='drier'")[obs_subset.stock50_100 > 0].stock50_100,
           decomp_df.query("MAP_cat=='drier'")[obs_subset.stock50_100 > 0].stock50_100,
           CLM_df.query("MAP_cat=='drier'")[obs_subset.stock50_100 > 0].stock50_100]

c30 = [podz_wo_subset.query("MAT_cat=='cooler'")[podz_wo_subset.stock30 > 0].stock30, obs_subset.query("MAT_cat=='cooler'").stock30,decomp_df.query("MAT_cat=='cooler'").stock30,CLM_df.query("MAT_cat=='cooler'").stock30]
wa30 = [podz_wo_subset.query("MAT_cat=='warmer'")[podz_wo_subset.stock30 > 0].stock30, obs_subset.query("MAT_cat=='warmer'").stock30,decomp_df.query("MAT_cat=='warmer'").stock30,CLM_df.query("MAT_cat=='warmer'").stock30]

        
c30_50 = [podz_wo_subset.query("MAT_cat=='cooler'")[podz_wo_subset.stock30_50 > 0].stock30_50 ,
          obs_subset.query("MAT_cat=='cooler'")[obs_subset.stock30_50 > 0].stock30_50,
          decomp_df.query("MAT_cat=='cooler'")[obs_subset.stock30_50 > 0].stock30_50,
          CLM_df.query("MAT_cat=='cooler'")[obs_subset.stock30_50 > 0].stock30_50]
          
wa30_50 = [podz_wo_subset.query("MAT_cat=='warmer'")[podz_wo_subset.stock30_50 > 0].stock30_50 ,
           obs_subset.query("MAT_cat=='warmer'")[obs_subset.stock30_50 > 0].stock30_50,
           decomp_df.query("MAT_cat=='warmer'")[obs_subset.stock30_50 > 0].stock30_50,
           CLM_df.query("MAT_cat=='warmer'")[obs_subset.stock30_50 > 0].stock30_50]

c50_100 = [podz_wo_subset.query("MAT_cat=='cooler'")[podz_wo_subset.stock50_100 > 0].stock50_100,
           obs_subset.query("MAT_cat=='cooler'")[obs_subset.stock50_100 > 0].stock50_100,
           decomp_df.query("MAT_cat=='cooler'")[obs_subset.stock50_100 > 0].stock50_100,
           CLM_df.query("MAT_cat=='cooler'")[obs_subset.stock50_100 > 0].stock50_100]
wa50_100 = [podz_wo_subset.query("MAT_cat=='warmer'")[podz_wo_subset.stock50_100 > 0].stock50_100,
            obs_subset.query("MAT_cat=='warmer'")[obs_subset.stock50_100 > 0].stock50_100,
            decomp_df.query("MAT_cat=='warmer'")[obs_subset.stock50_100 > 0].stock50_100,
            CLM_df.query("MAT_cat=='warmer'")[obs_subset.stock50_100 > 0].stock50_100]

font_dict={"fontsize":"x-large","fontweight":"bold"}
labels_1stline_MAP=["drier  wetter","drier  wetter","drier  wetter","drier  wetter"]
labels_1stline_MAT=["cooler  warmer","cooler  warmer","cooler  warmer","cooler  warmer"]
labels_2ndline=["\nOBS podzols","\nOBS","\nMIMICS+","\nCLM"]
positions1=[.5,2.5,4.5,6.5]
positions2=[1.1,3.1,5.1,7.1]
ticks_pos=[0.8,2.8,4.8,6.8,0.8,2.8,4.8,6.8]

fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(nrows=2, ncols=3, figsize=(25, 14))

ax1.boxplot(c30,positions=positions1,labels = labels_1stline_MAT,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[0],"markeredgecolor":"black"});
ax1.boxplot(wa30,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[1],"markeredgecolor":"black"});
ax1.set_title("(a)",fontsize = 15, loc = "left");
ax1.set_title("0–30 cm",fontsize = 15, loc = "center");
ax1.set_ylabel("$\mathrm{kgC m^{-2}}$",fontsize=15);
ax1.set_xticks(ticks_pos);
ax1.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(c30[0]))],y=c30[0], label = "N=%i"%len(c30[0]),color="gray",alpha=0.5)
ax1.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(wa30[0]))],y=wa30[0], label = "N=%i"%len(wa30[0]),color="gray",alpha=0.5,marker='^')
ax1.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(c30[1]))],y=c30[1], label = "N=%i"%len(c30[1]),color=colors[0],alpha=0.5)
ax1.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(wa30[1]))],y=wa30[1], label = "N=%i"%len(wa30[1]),color=colors[0],alpha=0.5,marker='^')
ax1.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(c30[2]))],y=c30[2], label = "N=%i"%len(c30[2]),color=colors[1],alpha=0.5)
ax1.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(wa30[2]))],y=wa30[2], label = "N=%i"%len(wa30[2]),color=colors[1],alpha=0.5,marker='^')
ax1.scatter(x=[positions1[3]+random.uniform(-0.1, 0.1) for i in range(len(c30[3]))],y=c30[3], label = "N=%i"%len(c30[3]),color=colors[2],alpha=0.5)
ax1.scatter(x=[positions2[3]+random.uniform(-0.1, 0.1) for i in range(len(wa30[3]))],y=wa30[3], label = "N=%i"%len(wa30[3]),color=colors[2],alpha=0.5,marker='^')
ax1.legend()

ax2.boxplot(c30_50,positions=positions1,labels = labels_1stline_MAT,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[0],"markeredgecolor":"black"});
ax2.boxplot(wa30_50,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[1],"markeredgecolor":"black"});
ax2.set_title("(b)", fontsize = 15, loc = "left")
ax2.set_title(" 30–50 cm ", fontsize = 15, loc = "center")
ax2.set_xticks(ticks_pos);
ax2.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(c30_50[0]))],y=c30_50[0], label = "N=%i"%len(c30_50[0]),color="gray",alpha=0.5)
ax2.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(wa30_50[0]))],y=wa30_50[0], label = "N=%i"%len(wa30_50[0]),color="gray",alpha=0.5,marker='^')
ax2.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(c30_50[1]))],y=c30_50[1], label = "N=%i"%len(c30_50[1]),color=colors[0],alpha=0.5)
ax2.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(wa30_50[1]))],y=wa30_50[1], label = "N=%i"%len(wa30_50[1]),color=colors[0],alpha=0.5,marker='^')
ax2.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(c30_50[2]))],y=c30_50[2], label = "N=%i"%len(c30_50[2]),color=colors[1],alpha=0.5)
ax2.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(wa30_50[2]))],y=wa30_50[2], label = "N=%i"%len(wa30_50[2]),color=colors[1],alpha=0.5,marker='^')
ax2.scatter(x=[positions1[3]+random.uniform(-0.1, 0.1) for i in range(len(c30_50[3]))],y=c30_50[3], label = "N=%i"%len(c30_50[3]),color=colors[2],alpha=0.5)
ax2.scatter(x=[positions2[3]+random.uniform(-0.1, 0.1) for i in range(len(wa30_50[3]))],y=wa30_50[3], label = "N=%i"%len(wa30_50[3]),color=colors[2],alpha=0.5,marker='^')
ax2.legend()

ax3.boxplot(c50_100,positions=positions1,vert=True,showmeans=True,labels = labels_1stline_MAT,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[0],"markeredgecolor":"black"});
ax3.boxplot(wa50_100,positions=positions2,vert=True,showmeans=True,labels = labels_2ndline,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":temp_colors[1],"markeredgecolor":"black"});
ax3.set_title("(c)",fontsize = 15, loc = "left");
ax3.set_title("50–100 cm",fontsize = 15, loc = "center");
ax3.set_xticks(ticks_pos);
ax3.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(c50_100[0]))],y=c50_100[0], label = "N=%i"%len(c50_100[0]),color="gray",alpha=0.5)
ax3.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(wa50_100[0]))],y=wa50_100[0], label = "N=%i"%len(wa50_100[0]),color="gray",alpha=0.5,marker='^')
ax3.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(c50_100[1]))],y=c50_100[1], label = "N=%i"%len(c50_100[1]),color=colors[0],alpha=0.5)
ax3.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(wa50_100[1]))],y=wa50_100[1], label = "N=%i"%len(wa50_100[1]),color=colors[0],alpha=0.5,marker='^')
ax3.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(c50_100[2]))],y=c50_100[2], label = "N=%i"%len(c50_100[2]),color=colors[1],alpha=0.5)
ax3.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(wa50_100[2]))],y=wa50_100[2], label = "N=%i"%len(wa50_100[2]),color=colors[1],alpha=0.5,marker='^')
ax3.scatter(x=[positions1[3]+random.uniform(-0.1, 0.1) for i in range(len(c50_100[3]))],y=c50_100[3], label = "N=%i"%len(c50_100[3]),color=colors[2],alpha=0.5)
ax3.scatter(x=[positions2[3]+random.uniform(-0.1, 0.1) for i in range(len(wa50_100[3]))],y=wa50_100[3], label = "N=%i"%len(wa50_100[3]),color=colors[2],alpha=0.5,marker='^')
ax3.legend()

ax4.boxplot(d30,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"});
ax4.boxplot(w30,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"});
ax4.set_title("(d)",fontsize = 15, loc = "left");
ax4.set_title("0–30 cm",fontsize = 15, loc = "center");
ax4.set_ylabel("$\mathrm{kgC m^{-2}}$",fontsize=16);
ax4.set_xticks(ticks_pos);
ax4.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(d30[0]))],y=d30[0], label = "N=%i"%len(d30[0]),color="gray",alpha=0.5)
ax4.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(w30[0]))],y=w30[0], label = "N=%i"%len(w30[0]),color="gray",alpha=0.5,marker='^')
ax4.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(d30[1]))],y=d30[1], label = "N=%i"%len(d30[1]),color=colors[0],alpha=0.5)
ax4.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(w30[1]))],y=w30[1], label = "N=%i"%len(w30[1]),color=colors[0],alpha=0.5,marker='^')
ax4.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(d30[2]))],y=d30[2], label = "N=%i"%len(d30[2]),color=colors[1],alpha=0.5)
ax4.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(w30[2]))],y=w30[2], label = "N=%i"%len(w30[2]),color=colors[1],alpha=0.5,marker='^')
ax4.scatter(x=[positions1[3]+random.uniform(-0.1, 0.1) for i in range(len(d30[3]))],y=d30[3], label = "N=%i"%len(d30[3]),color=colors[2],alpha=0.5)
ax4.scatter(x=[positions2[3]+random.uniform(-0.1, 0.1) for i in range(len(w30[3]))],y=w30[3], label = "N=%i"%len(w30[3]),color=colors[2],alpha=0.5,marker='^')
ax4.legend()

ax5.boxplot(d30_50,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"});
ax5.boxplot(w30_50,positions=positions2,labels =labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"});
ax5.set_title("(e)",fontsize = 15, loc = "left");
ax5.set_title("30–50 cm",fontsize = 15, loc = "center");
ax5.set_xticks(ticks_pos);
ax5.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(d30_50[0]))],y=d30_50[0], label = "N=%i"%len(d30_50[0]),color="gray",alpha=0.5)
ax5.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(w30_50[0]))],y=w30_50[0], label = "N=%i"%len(w30_50[0]),color="gray",alpha=0.5,marker='^')
ax5.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(d30_50[1]))],y=d30_50[1], label = "N=%i"%len(d30_50[1]),color=colors[0],alpha=0.5)
ax5.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(w30_50[1]))],y=w30_50[1], label = "N=%i"%len(w30_50[1]),color=colors[0],alpha=0.5,marker='^')
ax5.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(d30_50[2]))],y=d30_50[2], label = "N=%i"%len(d30_50[2]),color=colors[1],alpha=0.5)
ax5.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(w30_50[2]))],y=w30_50[2], label = "N=%i"%len(w30_50[2]),color=colors[1],alpha=0.5,marker='^')
ax5.scatter(x=[positions1[3]+random.uniform(-0.1, 0.1) for i in range(len(d30_50[3]))],y=d30_50[3], label = "N=%i"%len(d30_50[3]),color=colors[2],alpha=0.5)
ax5.scatter(x=[positions2[3]+random.uniform(-0.1, 0.1) for i in range(len(w30_50[3]))],y=w30_50[3], label = "N=%i"%len(w30_50[3]),color=colors[2],alpha=0.5,marker='^')
ax5.legend()

ax6.boxplot(d50_100,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"});
ax6.boxplot(w50_100,positions=positions2,labels =labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"});
ax6.set_title(r"$c)$"+"\t\t\t50-100 cm",fontsize = 15, loc = "left");
ax6.set_title(r"(f)",fontsize = 15, loc = "left");
ax6.set_title("50–100 cm",fontsize = 15, loc = "center");
ax6.set_xticks(ticks_pos);
ax6.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(d50_100[0]))],y=d50_100[0], label = "N=%i"%len(d50_100[0]),color="gray",alpha=0.5)
ax6.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(w50_100[0]))],y=w50_100[0], label = "N=%i"%len(w50_100[0]),color="gray",alpha=0.5,marker='^')
ax6.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(d50_100[1]))],y=d50_100[1], label = "N=%i"%len(d50_100[1]),color=colors[0],alpha=0.5)
ax6.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(w50_100[1]))],y=w50_100[1], label = "N=%i"%len(w50_100[1]),color=colors[0],alpha=0.5,marker='^')
ax6.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(d50_100[2]))],y=d50_100[2], label = "N=%i"%len(d50_100[2]),color=colors[1],alpha=0.5)
ax6.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(w50_100[2]))],y=w50_100[2], label = "N=%i"%len(w50_100[2]),color=colors[1],alpha=0.5,marker='^')
ax6.scatter(x=[positions1[3]+random.uniform(-0.1, 0.1) for i in range(len(d50_100[3]))],y=d50_100[3], label = "N=%i"%len(d50_100[3]),color=colors[2],alpha=0.5)
ax6.scatter(x=[positions2[3]+random.uniform(-0.1, 0.1) for i in range(len(w50_100[3]))],y=w50_100[3], label = "N=%i"%len(w50_100[3]),color=colors[2],alpha=0.5,marker='^')
ax6.legend()


plt.savefig("./figures/f06_include_podzols.pdf",bbox_inches="tight")


### f06 analysis

#### Difference between datasets, warmer and cooler:

In [ ]:


print("....Significance tests comparing datasets: ....")
print("------0-30cm------------------")
print("-----------warmer-------------")
significance_ttest(wa30[0],"OBS podzols 0-30cm warmer", wa30[1],"OBS 0-30cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30[0],"OBS podzols 0-30cm warmer", wa30[2],"MIMICS+ 0-30cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30[0],"OBS podzols 0-30cm warmer", wa30[3],"CLM 0-30cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30[1],"OBS 0-30cm warmer", wa30[2],"MIMICS+ 0-30cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30[1],"OBS 0-30cm warmer", wa30[3],"CLM 0-30cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30[3],"CLM 0-30cm warmer", wa30[2],"MIMICS+ 0-30cm warmer",alpha=0.05,equal_var=False)
print("-----------cooler-------------")
significance_ttest(c30[0],"OBS podzols 0-30cm cooler", c30[1],"OBS 0-30cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30[0],"OBS podzols 0-30cm cooler", c30[2],"MIMICS+ 0-30cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30[0],"OBS podzols 0-30cm cooler", c30[3],"CLM 0-30cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30[1],"OBS 0-30cm cooler", c30[2],"MIMICS+ 0-30cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30[1],"OBS 0-30cm cooler", c30[3],"CLM 0-30cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30[3],"CLM 0-30cm cooler", c30[2],"MIMICS+ 0-30cm cooler",alpha=0.05,equal_var=False)

print("------30-50cm------------------")
print("-----------warmer-------------")
significance_ttest(wa30_50[0],"OBS podzols 30-50cm warmer", wa30_50[1],"OBS 30-50cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30_50[0],"OBS podzols 30-50cm warmer", wa30_50[2],"MIMICS+ 30-50cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30_50[0],"OBS podzols 30-50cm warmer", wa30_50[3],"CLM 30-50cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30_50[1],"OBS 30-50cm warmer", wa30_50[2],"MIMICS+ 30-50cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30_50[1],"OBS 30-50cm warmer", wa30_50[3],"CLM 30-50cm warmer",alpha=0.05,equal_var=False)
significance_ttest(wa30_50[3],"CLM 30-50cm warmer", wa30_50[2],"MIMICS+ 30-50cm warmer",alpha=0.05,equal_var=False)
print("-----------cooler-------------")
significance_ttest(c30_50[0],"OBS podzols 30-50cm cooler", c30_50[1],"OBS 30-50cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30_50[0],"OBS podzols 30-50cm cooler", c30_50[2],"MIMICS+ 30-50cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30_50[0],"OBS podzols 30-50cm cooler", c30_50[3],"CLM 30-50cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30_50[1],"OBS 30-50cm cooler", c30_50[2],"MIMICS+ 30-50cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30_50[1],"OBS 30-50cm cooler", c30_50[3],"CLM 30-50cm cooler",alpha=0.05,equal_var=False)
significance_ttest(c30_50[3],"CLM 30-50cm cooler", c30_50[2],"MIMICS+ 30-50cm cooler",alpha=0.05,equal_var=False)

print("------50-100cm------------------")
print("-----------warmer-------------")
significance_ttest(wa50_100[0],"OBS podzols 50-100 warmer", wa50_100[1],"OBS 50-100 warmer",alpha=0.05,equal_var=False)
significance_ttest(wa50_100[0],"OBS podzols 50-100 warmer", wa50_100[2],"MIMICS+ 50-100 warmer",alpha=0.05,equal_var=False)
significance_ttest(wa50_100[0],"OBS podzols 50-100 warmer", wa50_100[3],"CLM 50-100 warmer",alpha=0.05,equal_var=False)
significance_ttest(wa50_100[1],"OBS 50-100 warmer", wa50_100[2],"MIMICS+ 50-100 warmer",alpha=0.05,equal_var=False)
significance_ttest(wa50_100[1],"OBS 50-100 warmer", wa50_100[3],"CLM 50-100 warmer",alpha=0.05,equal_var=False)
significance_ttest(wa50_100[3],"CLM 50-100 warmer", wa50_100[2],"MIMICS+ 50-100 warmer",alpha=0.05,equal_var=False)
print("-----------cooler-------------")
significance_ttest(c50_100[0],"OBS podzols 50-100 cooler", c50_100[1],"OBS 50-100 cooler",alpha=0.05,equal_var=False)
significance_ttest(c50_100[0],"OBS podzols 50-100 cooler", c50_100[2],"MIMICS+ 50-100 cooler",alpha=0.05,equal_var=False)
significance_ttest(c50_100[0],"OBS podzols 50-100 cooler", c50_100[3],"CLM 50-100 cooler",alpha=0.05,equal_var=False)
significance_ttest(c50_100[1],"OBS 50-100 cooler", c50_100[2],"MIMICS+ 50-100 cooler",alpha=0.05,equal_var=False)
significance_ttest(c50_100[1],"OBS 50-100 cooler", c50_100[3],"CLM 50-100 cooler",alpha=0.05,equal_var=False)
significance_ttest(c50_100[3],"CLM 50-100 cooler", c50_100[2],"MIMICS+ 50-100 cooler",alpha=0.05,equal_var=False)

#### Difference between datasets drier and wetter

In [ ]:
print("....Significance tests comparing datasets: ....")
print("------0-30cm------------------")
print("-----------drier-------------")
mf.significance_ttest(d30[1],"OBS 0-30cm drier", d30[2],"MIMICS+ 0-30cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d30[1],"OBS 0-30cm drier", d30[3],"CLM 0-30cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d30[3],"CLM 0-30cm drier", d30[2],"MIMICS+ 0-30cm drier",alpha=0.05,equal_var=False)

mf.significance_ttest(d30[0],"OBS podzols 0-30cm drier", d30[2],"MIMICS+ 0-30cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d30[0],"OBS podzols 0-30cm drier", d30[3],"CLM 0-30cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d30[0],"OBS podzols 0-30cm drier", d30[1],"OBS 0-30cm drier",alpha=0.05,equal_var=False)

print("-----------wetter-------------")
mf.significance_ttest(w30[1],"OBS 0-30cm wetter", w30[2],"MIMICS+ 0-30cm wetter",alpha=0.05,equal_var=False)
mf.significance_ttest(w30[1],"OBS 0-30cm wetter", w30[3],"CLM 0-30cm wetter",alpha=0.05,equal_var=False)
mf.significance_ttest(w30[3],"CLM 0-30cm wetter", w30[2],"MIMICS+ 0-30cm wetter",alpha=0.05,equal_var=False)

mf.significance_ttest(w30[0],"OBS podzols 0-30cm drier", w30[2],"MIMICS+ 0-30cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(w30[0],"OBS podzols 0-30cm drier", w30[3],"CLM 0-30cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(w30[0],"OBS podzols 0-30cm drier", w30[1],"OBS 0-30cm drier",alpha=0.05,equal_var=False)

print("------30-50cm------------------")
print("-----------drier-------------")
mf.significance_ttest(d30_50[1],"OBS 30-50 drier", d30_50[2],"MIMICS+ 30-50 drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d30_50[1],"OBS 30-50 drier", d30_50[3],"CLM 30-50 drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d30_50[3],"CLM 30-50 drier", d30_50[2],"MIMICS+ 30-50 drier",alpha=0.05,equal_var=False)

mf.significance_ttest(d30_50[0],"OBS podzols 30-50 drier", d30_50[2],"MIMICS+ 30-50 drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d30_50[0],"OBS podzols 30-50 drier", d30_50[3],"CLM 30-50 drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d30_50[0],"OBS podzols 30-50 drier", d30_50[1],"OBS 30-50 drier",alpha=0.05,equal_var=False)

print("-----------wetter-------------")
mf.significance_ttest(w30_50[1],"OBS 30-50 wetter", w30_50[2],"MIMICS+ 30-50 wetter",alpha=0.05,equal_var=False)
mf.significance_ttest(w30_50[1],"OBS 30-50 wetter", w30_50[3],"CLM 30-50 wetter",alpha=0.05,equal_var=False)
mf.significance_ttest(w30_50[3],"CLM 30-50 wetter", w30_50[2],"MIMICS+ 30-50 wetter",alpha=0.05,equal_var=False)

mf.significance_ttest(w30_50[0],"OBS podzols 30-50 drier", w30_50[2],"MIMICS+ 30-50 drier",alpha=0.05,equal_var=False)
mf.significance_ttest(w30_50[0],"OBS podzols 30-50 drier", w30_50[3],"CLM 30-50 drier",alpha=0.05,equal_var=False)
mf.significance_ttest(w30_50[0],"OBS podzols 30-50 drier", w30_50[1],"OBS 30-50 drier",alpha=0.05,equal_var=False)

print("------50-100cm------------------")
print("-----------drier-------------")
mf.significance_ttest(d50_100[1],"OBS 50-100cm drier", d50_100[2],"MIMICS+ 50-100cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d50_100[1],"OBS 50-100cm drier", d50_100[3],"CLM 50-100cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d50_100[3],"CLM 50-100cm drier", d50_100[2],"MIMICS+ 50-100cm drier",alpha=0.05,equal_var=False)

mf.significance_ttest(d50_100[0],"OBS podzols 50-100cm drier", d50_100[2],"MIMICS+ 50-100cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d50_100[0],"OBS podzols 50-100cm drier", d50_100[3],"CLM 50-100cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(d50_100[0],"OBS podzols 50-100cm drier", d50_100[1],"OBS 50-100cm drier",alpha=0.05,equal_var=False)

print("-----------wetter-------------")
mf.significance_ttest(w50_100[1],"OBS 50-100cm wetter", w50_100[2],"MIMICS+ 50-100cm wetter",alpha=0.05,equal_var=False)
mf.significance_ttest(w50_100[1],"OBS 50-100cm wetter", w50_100[3],"CLM 50-100cm wetter",alpha=0.05,equal_var=False)
mf.significance_ttest(w50_100[3],"CLM 50-100cm wetter", w50_100[2],"MIMICS+ 50-100cm wetter",alpha=0.05,equal_var=False)

mf.significance_ttest(w50_100[0],"OBS podzols 50-100cm drier", w50_100[2],"MIMICS+ 50-100cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(w50_100[0],"OBS podzols 50-100cm drier", w50_100[3],"CLM 50-100cm drier",alpha=0.05,equal_var=False)
mf.significance_ttest(w50_100[0],"OBS podzols 50-100cm drier", w50_100[1],"OBS 50-100cm drier",alpha=0.05,equal_var=False)


#### Difference between drier/wetter and warmer/colder within a dataset

In [ ]:
print("-----OBS podzols cooler/warmer: ------\n")
mf.significance_ttest(wa30[0],"OBS podzols 0-30cm warmer",c30[0], "OBS podzols 0-30cm cooler",alpha=0.05,equal_var=True)
mf.significance_ttest(wa30_50[0],"OBS podzols 30-50cm warmer",c30_50[0], "OBS podzols 30-50cm cooler",alpha=0.05,equal_var=True)
mf.significance_ttest(wa50_100[0],"OBS podzols 50-100cm warmer",c50_100[0], "OBS podzols 50-100cm cooler",alpha=0.05,equal_var=True)

print("-----OBS podzols drier/wetter: ------\n")
mf.significance_ttest(w30[0],"OBS podzols 0-30cm wetter",d30[0], "OBS podzols 0-30cm drier",alpha=0.05,equal_var=True)
mf.significance_ttest(w30_50[0],"OBS podzols 30-50cm wetter",d30_50[0], "OBS podzols 30-50cm drier",alpha=0.05,equal_var=True)
mf.significance_ttest(w50_100[0],"OBS podzols 50-100cm wetter",d50_100[0], "OBS podzols 50-100cm drier",alpha=0.05,equal_var=True)


print("-----OBS cooler/warmer: ------\n")
mf.significance_ttest(wa30[1],"OBS 0-30cm warmer",c30[1], "OBS 0-30cm cooler",alpha=0.05,equal_var=True)
mf.significance_ttest(wa30_50[1],"OBS 30-50cm warmer",c30_50[1], "OBS 30-50cm cooler",alpha=0.05,equal_var=True)
mf.significance_ttest(wa50_100[1],"OBS 50-100cm warmer",c50_100[1], "OBS 50-100cm cooler",alpha=0.05,equal_var=True)

print("-----OBS drier/wetter: ------\n")
mf.significance_ttest(w30[1],"OBS 0-30cm wetter",d30[1], "OBS 0-30cm drier",alpha=0.05,equal_var=True)
mf.significance_ttest(w30_50[1],"OBS 30-50cm wetter",d30_50[1], "OBS 30-50cm drier",alpha=0.05,equal_var=True)
mf.significance_ttest(w50_100[1],"OBS 50-100cm wetter",d50_100[1], "OBS 50-100cm drier",alpha=0.05,equal_var=True)

print("-----MIMICS+ cooler/warmer: ------\n")
mf.significance_ttest(wa30[2],"MIMICS+ 0-30cm warmer",c30[2], "MIMICS+ 0-30cm cooler",alpha=0.05,equal_var=True)
mf.significance_ttest(wa30_50[2],"MIMICS+ 30-50cm warmer",c30_50[2], "MIMICS+ 30-50cm cooler",alpha=0.05,equal_var=True)
mf.significance_ttest(wa50_100[2],"MIMICS+ 50-100cm warmer",c50_100[2], "MIMICS+ 50-100cm cooler",alpha=0.05,equal_var=True)

print("-----MIMICS+ drier/wetter: ------\n")
mf.significance_ttest(w30[2],"MIMICS+ 0-30cm wetter",d30[2], "MIMICS+ 0-30cm drier",alpha=0.05,equal_var=True)
mf.significance_ttest(w30_50[2],"MIMICS+ 30-50cm wetter",d30_50[2], "MIMICS+ 30-50cm drier",alpha=0.05,equal_var=True)
mf.significance_ttest(w50_100[2],"MIMICS+ 50-100cm wetter",d50_100[2], "MIMICS+ 50-100cm drier",alpha=0.05,equal_var=True)

print("-----CLM cooler/warmer: ------\n")
mf.significance_ttest(wa30[3],"CLM 0-30cm warmer",c30[3], "CLM 0-30cm cooler",alpha=0.05,equal_var=True)
mf.significance_ttest(wa30_50[3],"CLM 30-50cm warmer",c30_50[3], "CLM 30-50cm cooler",alpha=0.05,equal_var=True)
mf.significance_ttest(wa50_100[3],"CLM 50-100cm warmer",c50_100[3], "CLM 50-100cm cooler",alpha=0.05,equal_var=True)

print("-----CLM drier/wetter: ------\n")
mf.significance_ttest(w30[3],"CLM 0-30cm wetter",d30[3], "CLM 0-30cm drier",alpha=0.05,equal_var=True)
mf.significance_ttest(w30_50[3],"CLM 30-50cm wetter",d30_50[3], "CLM 30-50cm drier",alpha=0.05,equal_var=True)
mf.significance_ttest(w50_100[3],"CLM 50-100cm wetter",d50_100[3], "CLM 50-100cm drier",alpha=0.05,equal_var=True)

#### fig:clim_profile_soil_water
Quartile plots using soil moisture instead of MAP as indicator of wet/dry conditions

In [ ]:
sns.set_style("darkgrid")
ws30 = [obs_subset.query("cat_SW=='wetter_soil'").stock30,decomp_df.query("cat_SW=='wetter_soil'").stock30,CLM_df.query("cat_SW=='wetter_soil'").stock30]
ds30 = [obs_subset.query("cat_SW=='drier_soil'").stock30,decomp_df.query("cat_SW=='drier_soil'").stock30,CLM_df.query("cat_SW=='drier_soil'").stock30]
ws30_50 = [obs_subset.query("cat_SW=='wetter_soil'")[obs_subset.stock30_50 > 0].stock30_50,
           decomp_df.query("cat_SW=='wetter_soil'")[obs_subset.stock30_50 > 0].stock30_50,
           CLM_df.query("cat_SW=='wetter_soil'")[obs_subset.stock30_50 > 0].stock30_50]
ds30_50 = [obs_subset.query("cat_SW=='drier_soil'")[obs_subset.stock30_50 > 0].stock30_50
           ,decomp_df.query("cat_SW=='drier_soil'")[obs_subset.stock30_50 > 0].stock30_50,CLM_df.query("cat_SW=='drier_soil'")[obs_subset.stock30_50 > 0].stock30_50]
ws50_100 = [obs_subset.query("cat_SW=='wetter_soil'")[obs_subset.stock50_100 > 0].stock50_100,
            decomp_df.query("cat_SW=='wetter_soil'")[obs_subset.stock50_100 > 0].stock50_100,
            CLM_df.query("cat_SW=='wetter_soil'")[obs_subset.stock50_100 > 0].stock50_100]
ds50_100 = [obs_subset.query("cat_SW=='drier_soil'")[obs_subset.stock50_100 > 0].stock50_100,
            decomp_df.query("cat_SW=='drier_soil'")[obs_subset.stock50_100 > 0].stock50_100,
            CLM_df.query("cat_SW=='drier_soil'")[obs_subset.stock50_100 > 0].stock50_100]
ticks_pos=[0.8,2.8,4.8,0.8,2.8,4.8]
positions1=[.5,2.5,4.5]
positions2=[1.1,3.1,5.1]


font_dict={"fontsize":"x-large","fontweight":"bold"}
fig, ((ax1,ax2,ax3)) = plt.subplots(nrows=1, ncols=3, figsize=(25, 7))
fig.suptitle("Distributions sorted by soil moisture")
ax1.boxplot(ds30,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"beige","markeredgecolor":"black"});
ax1.boxplot(ws30,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"green","markeredgecolor":"black"});
ax1.set_title("0-30 cm");ax1.set_ylabel("[$kg/m^2$]");
#ax1.text(x=5,y=80,s="a)",fontdict=font_dict)
ax1.set_xticks(ticks_pos);
ax1.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(ds30[0]))],y=ds30[0], label = "N=%i"%len(ds30[0]),color=colors[0],alpha=0.4)
ax1.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(ws30[0]))],y=ws30[0], label = "N=%i"%len(ws30[0]),color=colors[0],alpha=0.4,marker='^')
ax1.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(ds30[1]))],y=ds30[1], label = "N=%i"%len(ds30[1]),color=colors[1],alpha=0.4)
ax1.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(ws30[1]))],y=ws30[1], label = "N=%i"%len(ws30[1]),color=colors[1],alpha=0.4,marker='^')
ax1.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(ds30[2]))],y=ds30[2], label = "N=%i"%len(ds30[2]),color=colors[2],alpha=0.4)
ax1.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(ws30[2]))],y=ws30[2], label = "N=%i"%len(ws30[2]),color=colors[2],alpha=0.4,marker='^')
ax1.legend()
ax1.set_title(r"(a)",fontsize = 15, loc = "left");

ax2.boxplot(ds30_50,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"beige","markeredgecolor":"black"});
ax2.boxplot(ws30_50,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"green","markeredgecolor":"black"});
ax2.set_title("30-50 cm");ax2.set_ylabel("[$kg/m^2$]");
ax2.set_title(r"(b)",fontsize = 15, loc = "left");

#ax2.text(x=5,y=42,s="b)",fontdict=font_dict)
ax2.set_xticks(ticks_pos);
ax2.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(ds30_50[0]))],y=ds30_50[0], label = "N=%i"%len(ds30_50[0]),color=colors[0],alpha=0.4)
ax2.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(ws30_50[0]))],y=ws30_50[0], label = "N=%i"%len(ws30_50[0]),color=colors[0],alpha=0.4,marker='^')
ax2.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(ds30_50[1]))],y=ds30_50[1], label = "N=%i"%len(ds30_50[1]),color=colors[1],alpha=0.4)
ax2.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(ws30_50[1]))],y=ws30_50[1], label = "N=%i"%len(ws30_50[1]),color=colors[1],alpha=0.4,marker='^')
ax2.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(ds30_50[2]))],y=ds30_50[2], label = "N=%i"%len(ds30_50[2]),color=colors[2],alpha=0.4)
ax2.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(ws30_50[2]))],y=ws30_50[2], label = "N=%i"%len(ws30_50[2]),color=colors[2],alpha=0.4,marker='^')
ax2.legend()

ax3.boxplot(ds50_100,positions=positions1,labels = labels_1stline_MAP,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"beige","markeredgecolor":"black"});
ax3.boxplot(ws50_100,positions=positions2,labels = labels_2ndline,vert=True,showmeans=True,
medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":"green","markeredgecolor":"black"});
ax3.set_ylabel("[$kg/m^2$]");ax3.set_title("50_100 cm");
ax3.set_title(r"(c)",fontsize = 15, loc = "left");

#ax3.text(x=5,y=18,s="c)",fontdict=font_dict)
ax3.set_xticks(ticks_pos);
ax3.scatter(x=[positions1[0]+random.uniform(-0.1, 0.1) for i in range(len(ds50_100[0]))],y=ds50_100[0], label = "N=%i"%len(ds50_100[0]),color=colors[0],alpha=0.4)
ax3.scatter(x=[positions2[0]+random.uniform(-0.1, 0.1) for i in range(len(ws50_100[0]))],y=ws50_100[0], label = "N=%i"%len(ws50_100[0]),color=colors[0],alpha=0.4,marker='^')
ax3.scatter(x=[positions1[1]+random.uniform(-0.1, 0.1) for i in range(len(ds50_100[1]))],y=ds50_100[1], label = "N=%i"%len(ds50_100[1]),color=colors[1],alpha=0.4)
ax3.scatter(x=[positions2[1]+random.uniform(-0.1, 0.1) for i in range(len(ws50_100[1]))],y=ws50_100[1], label = "N=%i"%len(ws50_100[1]),color=colors[1],alpha=0.4,marker='^')
ax3.scatter(x=[positions1[2]+random.uniform(-0.1, 0.1) for i in range(len(ds50_100[2]))],y=ds50_100[2], label = "N=%i"%len(ds50_100[2]),color=colors[2],alpha=0.4)
ax3.scatter(x=[positions2[2]+random.uniform(-0.1, 0.1) for i in range(len(ws50_100[2]))],y=ws50_100[2], label = "N=%i"%len(ws50_100[2]),color=colors[2],alpha=0.4,marker='^')
ax3.legend()
plt.savefig("./figures/climate_profile_soilwater.pdf",bbox_inches="tight")

#del ws30,ws30_50,ws50_100,ds30,ds30_50,ds50_100

In [ ]:
mf.significance_ttest(ds30[0],"obs subset drier_soil 30cm",ws30[0], "obs subset wetter_soil, 30cm",alpha=0.050,equal_var=False)
mf.significance_ttest(ds30_50[0],"obs subset drier_soil 30_50cm",ws30_50[0], "obs subset wetter_soil, 30_50cm",alpha=0.050,equal_var=False)
mf.significance_ttest(ds50_100[0],"obs subset drier_soil 50_100cm",ws50_100[0], "obs subset wetter_soil, 50_100cm",alpha=0.050,equal_var=False)
print("----")
mf.significance_ttest(ds30[1],"MIMICS+ drier_soil 30cm",ws30[1], "MIMICS+ wetter_soil, 30cm",alpha=0.050,equal_var=False)
mf.significance_ttest(ds30_50[1],"MIMICS+ drier_soil 30_50cm",ws30_50[1], "MIMICS+ wetter_soil, 30_50cm",alpha=0.050,equal_var=False)
mf.significance_ttest(ds50_100[1],"MIMICS+ drier_soil 50_100cm",ws50_100[1], "MIMICS+ wetter_soil, 50_100cm",alpha=0.050,equal_var=False)

print("----")
mf.significance_ttest(ds30[2],"CLM drier_soil 30cm",ws30[2], "CLM wetter_soil, 30cm",alpha=0.050,equal_var=False)
mf.significance_ttest(ds30_50[2],"CLM drier_soil 30_50cm",ws30_50[2], "CLM wetter_soil, 30_50cm",alpha=0.050,equal_var=False)
mf.significance_ttest(ds50_100[2],"CLM drier_soil 50_100cm",ws50_100[2], "CLM wetter_soil, 50_100cm",alpha=0.050,equal_var=False)



### Yearly input plots and tables

In [ ]:
input_df = pd.DataFrame(columns = ["SITE","Litter input C","Litter input N","N deposition","C alloc to myc"])

for ds in decomp_data:
    #CARBON
    inp = ((ds.C_PlantLITm+ds.C_PlantLITs+ds.C_PlantSOMc+ds.C_PlantSOMp).T*depth[0:mf.no_of_layers(ds)]).sum("levsoi") #Sum mass input throughout column
    mean_input = (inp*24).groupby("time.year").sum()[:-1].mean() #[gC/m2h]*24 --> [gC/m2 day] input per day. Group values into separate years, summarize input in each year (skip 1993) and return the mean value of the 5 years

    inp_myc = ((ds.C_PlantEcM+ds.C_PlantAM).T*depth[0:mf.no_of_layers(ds)]).sum("levsoi") #Sum mass input throughout column
    mean_input_myc = (inp_myc*24).groupby("time.year").sum()[:-1].mean() #[gC/m2h]*24 --> [gC/m2] input per day. Group values into separate years, summarize input in each year (skip 1993) and return the mean value of the 5 years
    
    #NITROGEN
    inp_N = ((ds.N_PlantLITm+ds.N_PlantLITs+ds.N_PlantSOMc+ds.N_PlantSOMp).T*depth[0:mf.no_of_layers(ds)]).sum("levsoi")
    mean_input_N=(inp_N*24).groupby("time.year").sum()[:-1].mean()

    inp_Ndep = ((ds.N_Deposition).T*depth[0:mf.no_of_layers(ds)]).sum("levsoi")
    mean_input_Ndep=(inp_Ndep*24).groupby("time.year").sum()[:-1].mean()
    input_df.loc[len(input_df)]={"SITE":ds.site_name,"Litter input C":float(mean_input.values),"Litter input N":float(mean_input_N),"C alloc to myc":float(mean_input_myc),"N deposition": float(mean_input_Ndep)}
input_df

In [ ]:
fig, ax = plt.subplots(figsize = [8,8])
plt.scatter(input_df["Litter input C"],input_df["Litter input N"],label = "Litter input only")
plt.scatter(input_df["Litter input C"]+input_df["C alloc to myc"],input_df["Litter input N"]+input_df["N deposition"],label="All input")
ax.set_xlabel("Total C input [$gC m^{-2} y^{-1}$]");ax.set_ylabel("Total N input [$gN m^{-2} y^{-1}$]")
ax.legend();
fig.savefig("./figures/input_distribution.pdf",bbox_inches="tight")


### fig:no_temp_dependence 

In [ ]:
#Sites between the black lines are included in the analysis
plt.figure(figsize=[20,7])
(clean_pod.sort_values("MAT").reset_index()["MAT"]).plot();
plt.plot([i for i in range(578)],[0 for i in range(578)],color="black")
plt.plot([i for i in range(578)],[5 for i in range(578)],color="black")
plt.xlabel("SITE NR");plt.ylabel("Temp");

In [ ]:
intervals = ["a","b","c","d","e","f","g","h","i","j"]
grouped_pod = clean_pod.groupby(pd.cut(clean_pod["MAT"], np.arange(0, 5.5, 0.5),labels=intervals))
grouped_pod.count()["MAT"]

In [ ]:

int = ["(0.0, 0.5]", "(0.5, 1.0]","(1.0, 1.5]","(1.5, 2.0]","(2.0, 2.5]","(2.5, 3.0]","(3.0, 3.5]","(3.5, 4.0]","(4.0, 4.5]","(4.5, 5.0]"]
fig, axes = plt.subplots(nrows=6,ncols=5,figsize=[23,25]);

axes = axes.ravel()
for i in range(10):
    df=grouped_pod.get_group(intervals[i])
    df_dry = df[df.MAP <= df.MAP.median()].stock30
    df_wet = df[df.MAP > df.MAP.median()].stock30
    axes[i].boxplot(df_dry,positions=[0.5],labels = ["dry"],vert=True,showmeans=True,
    medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"},whis=[0, 100]);
    axes[i].boxplot(df_wet,labels = ["wet"],vert=True,showmeans=True,
    medianprops={"color":"black"},positions=[0.8],meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"},whis=[0, 100]);
    axes[i].set_title(int[i]+" deg C",fontsize = 12, loc = "left");
    axes[i].set_title("0-30cm",fontsize = 12, loc = "right");
    axes[i].scatter(x=[0.5+random.uniform(-0.1, 0.1) for i in range(len(df_dry))],y=df_dry,marker="^", label = "N=%i"%len(df_dry),color=prec_colors[0],alpha=0.5)
    axes[i].scatter(x=[0.8+random.uniform(-0.1, 0.1) for i in range(len(df_wet))],y=df_wet, label = "N=%i"%len(df_wet),color=prec_colors[1],alpha=0.5)
    axes[i].legend();

for i in range(10,20):
    df=grouped_pod.get_group(intervals[i-10])
    df_dry = df[df.MAP <= df.MAP.median()][df.stock30_50 > 0].stock30_50
    df_wet = df[df.MAP > df.MAP.median()][df.stock30_50 > 0].stock30_50
    axes[i].boxplot(df_dry,positions=[0.5],labels = ["dry"],vert=True,showmeans=True,
    medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"},whis=[0, 100]);
    axes[i].boxplot(df_wet,labels = ["wet"],vert=True,showmeans=True,
    medianprops={"color":"black"},positions=[0.8],meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"},whis=[0, 100]);
    axes[i].set_title(int[i-10]+" deg C",fontsize = 12, loc = "left");
    axes[i].set_title("30-50 cm",fontsize = 12, loc = "right");
    # axes[i].set_xticks(ticks_pos);
    axes[i].scatter(x=[0.5+random.uniform(-0.1, 0.1) for i in range(len(df_dry))],y=df_dry,marker="^", label = "N=%i"%len(df_dry),color=prec_colors[0],alpha=0.5)
    axes[i].scatter(x=[0.8+random.uniform(-0.1, 0.1) for i in range(len(df_wet))],y=df_wet, label = "N=%i"%len(df_wet),color=prec_colors[1],alpha=0.5)
    axes[i].legend();


for i in range(20,30):
    df=grouped_pod.get_group(intervals[i-20])
    df_dry = df[df.MAP <= df.MAP.median()][df.stock50_100 > 0].stock50_100
    df_wet = df[df.MAP > df.MAP.median()][df.stock50_100 > 0].stock50_100
    axes[i].boxplot(df_dry,positions=[0.5],labels = ["dry"],vert=True,showmeans=True,
    medianprops={"color":"black"},meanprops={"marker":"D","markerfacecolor":prec_colors[0],"markeredgecolor":"black"},whis=[0, 100]);
    axes[i].boxplot(df_wet,labels = ["wet"],vert=True,showmeans=True,
    medianprops={"color":"black"},positions=[0.8],meanprops={"marker":"D","markerfacecolor":prec_colors[1],"markeredgecolor":"black"},whis=[0, 100]);
    axes[i].set_title(int[i-20]+" deg C",fontsize = 12, loc = "left");
    axes[i].set_title("50-100 cm",fontsize = 12, loc = "right");
    # axes[i].set_xticks(ticks_pos);
    axes[i].scatter(x=[0.5+random.uniform(-0.1, 0.1) for i in range(len(df_dry))],y=df_dry,marker="^", label = "N=%i"%len(df_dry),color=prec_colors[0],alpha=0.5)
    axes[i].scatter(x=[0.8+random.uniform(-0.1, 0.1) for i in range(len(df_wet))],y=df_wet, label = "N=%i"%len(df_wet),color=prec_colors[1],alpha=0.5)
    axes[i].legend();

axes[0].set_ylabel("$kgCm^{-2}$")
axes[5].set_ylabel("$kgCm^{-2}$")
axes[10].set_ylabel("$kgCm^{-2}$")
axes[15].set_ylabel("$kgCm^{-2}$")
axes[20].set_ylabel("$kgCm^{-2}$")
axes[25].set_ylabel("$kgCm^{-2}$");
#fig.savefig("./figures/no_temp_dependence.pdf",bbox_inches="tight")#

## fig:scatterplot

In [ ]:
fig,axes=plt.subplots(1,3,figsize = [25,7])

#MIMICS+
axes[0].scatter(x= obs_subset.stock30,y= decomp_df.stock30,color=colors[0])
z = np.polyfit(obs_subset.stock30, decomp_df.stock30, 1)
p = np.poly1d(z)
R2 = np.corrcoef(decomp_df.stock30,p(obs_subset.stock30))[0,1]**2
#add trendline to plot
axes[0].plot(obs_subset.stock30, p(obs_subset.stock30),label="MIMICS+: "+str(p)+", $R^2=$"+str(round(R2,2)),color=colors[0])


#CLM:
axes[0].scatter(x= obs_subset.stock30,y= CLM_df.stock30,color=colors[1])
z = np.polyfit(obs_subset.stock30, CLM_df.stock30, 1)
p = np.poly1d(z)
R2 = np.corrcoef(CLM_df.stock30,p(obs_subset.stock30))[0,1]**2

#add trendline to plot
axes[0].plot(obs_subset.stock30, p(obs_subset.stock30),label="CLM: "+str(p)+", $R^2=$"+str(round(R2,2)),color=colors[1])

axes[0].legend();axes[0].set_xlabel("obs C stock [$kg/m2$]");axes[0].set_ylabel("modeled C stock [$kg/m2$]");axes[0].set_title("0-30cm")

#30-50cm
axes[1].scatter(x= obs_subset[obs_subset.stock30_50 > 0].stock30_50,y= decomp_df[obs_subset.stock30_50 > 0].stock30_50,color=colors[0])
z = np.polyfit(obs_subset[obs_subset.stock30_50 > 0].stock30_50, decomp_df[obs_subset.stock30_50 > 0].stock30_50, 1)
p = np.poly1d(z)
R2 = np.corrcoef(decomp_df[obs_subset.stock30_50 > 0].stock30_50,p(obs_subset[obs_subset.stock30_50 > 0].stock30_50))[0,1]**2
#add trendline to plot
axes[1].plot(obs_subset[obs_subset.stock30_50 > 0].stock30_50, p(obs_subset[obs_subset.stock30_50 > 0].stock30_50),label="MIMICS+: "+str(p)+", $R^2=$"+str(round(R2,2)),color=colors[0])

axes[1].scatter(x= obs_subset[obs_subset.stock30_50 > 0].stock30_50,y= CLM_df[obs_subset.stock30_50 > 0].stock30_50,color=colors[1])
z = np.polyfit(obs_subset[obs_subset.stock30_50 > 0].stock30_50, CLM_df[obs_subset.stock30_50 > 0].stock30_50, 1)
p = np.poly1d(z)
R2 = np.corrcoef(CLM_df[obs_subset.stock30_50 > 0].stock30_50,p(obs_subset[obs_subset.stock30_50 > 0].stock30_50))[0,1]**2
#add trendline to plot
axes[1].plot(obs_subset[obs_subset.stock30_50 > 0].stock30_50, p(obs_subset[obs_subset.stock30_50 > 0].stock30_50),label="CLM: "+str(p)+", $R^2=$"+str(round(R2,2)),color=colors[1])
axes[1].legend();axes[1].set_xlabel("obs C stock [$kg/m2$]");axes[1].set_ylabel("modeled C stock [$kg/m2$]");axes[1].set_title("30-50cm")


#50-100cm
axes[2].scatter(x= obs_subset[(obs_subset.stock50_100 > 0)].stock50_100,y= decomp_df[obs_subset.stock50_100 > 0].stock50_100,color=colors[0])
z = np.polyfit(obs_subset[(obs_subset.stock50_100 > 0)].stock50_100, decomp_df[(obs_subset.stock50_100 > 0)].stock50_100, 1)
p = np.poly1d(z)
R2 = np.corrcoef(decomp_df[obs_subset.stock50_100 > 0].stock50_100,p(obs_subset[obs_subset.stock50_100 > 0].stock50_100))[0,1]**2
#add trendline to plot
axes[2].plot(obs_subset[(obs_subset.stock50_100 > 0)].stock50_100, p(obs_subset[(obs_subset.stock50_100 > 0)].stock50_100),label="MIMICS+: "+str(p)+", $R^2=$"+str(round(R2,2)),color=colors[0])

axes[2].scatter(x= obs_subset[(obs_subset.stock50_100 > 0)].stock50_100,y= CLM_df[(obs_subset.stock50_100 > 0)].stock50_100,color=colors[1])
z = np.polyfit(obs_subset[(obs_subset.stock50_100 > 0)].stock50_100, CLM_df[(obs_subset.stock50_100 > 0)].stock50_100, 1)
p = np.poly1d(z)
R2 = np.corrcoef(CLM_df[obs_subset.stock50_100 > 0].stock50_100,p(obs_subset[obs_subset.stock50_100 > 0].stock50_100))[0,1]**2
#add trendline to plot
axes[2].plot(obs_subset[(obs_subset.stock50_100 > 0)].stock50_100, p(obs_subset[(obs_subset.stock50_100 > 0)].stock50_100),label="CLM: "+str(p)+", $R^2=$"+str(round(R2,2)),color=colors[1])

axes[2].legend();axes[2].set_xlabel("obs C stock [$kg/m2$]");axes[2].set_ylabel("modeled C stock [$kg/m2$]");axes[2].set_title("50-100cm")

fig.savefig("./figures/scatterplot.pdf",bbox_inches="tight")